# BedControl最適化

本問題では、target_date = "2021/8/21" が最大になるようにinputデータを用意している。  
ただし、計算量が大きくメモリ不足になるため、このNotebookではtarget_date = "2021/8/10"で計算している。  
また、計算量削減のため、PyQuboは利用していない。  

転院予定者と空きベットについてのマッチング

# データを読み込む

In [3]:
#from google.colab import drive
#drive.mount('/content/drive')

In [4]:
import pandas as pd
import numpy as np
import datetime

#関数で引数をリスト内の値で限定するためにLiteralを利用するが、Pythonのバージョンによってパッケージが異なるので特殊な呼び出し方をしている
try:
  from typing import Literal
except ImportError:
  from typing_extensions import Literal

## 病院情報をDB_Hospitalsに読み込み

In [5]:
DB_Hospitals = pd.read_csv(u'.\\input\\DB_Hospitals.csv', encoding='utf-8')
DB_Hospitals = DB_Hospitals.set_index('hospital_id')
DB_Hospitals.head(3)

,医療機関名称,全体,高度急性期,急性期,回復期,慢性期,休棟中（今後再開予定）,休棟中（今後廃止予定）,介護保険施設等,hospiatl_address,tel,三次医療圏,二次医療圏
hospital_id,,,,,,,,,,,,,
H00001,貝山中央病院,60,0,0,0,60,0,0,0,NaN,NaN,宮城県全域,仙台医療圏
H00002,早坂愛生会病院,52,0,0,0,52,0,0,0,NaN,NaN,宮城県全域,仙台医療圏
H00003,東北大学病院,1165,713,405,0,0,0,47,0,NaN,NaN,宮城県全域,仙台医療圏


## 空きベッド情報をdb_empty_bedsに読み込み

In [6]:
import os

In [7]:
# 病院の空きベッド情報のファイルリスト作成
filepath = u'.\\input\\'
# filelist = ["H00001_Beds.csv","H00002_Beds.csv","H00003_Beds.csv"]
filelist = [b for b in os.listdir(filepath) if '_Beds.csv' in b]
filelist = [filepath + f for f in filelist]
filelist

['.\\input\\H00001_Beds.csv',
 '.\\input\\H00002_Beds.csv',
 '.\\input\\H00003_Beds.csv',
 '.\\input\\H00004_Beds.csv',
 '.\\input\\H00005_Beds.csv',
 '.\\input\\H00006_Beds.csv',
 '.\\input\\H00007_Beds.csv',
 '.\\input\\H00008_Beds.csv',
 '.\\input\\H00009_Beds.csv',
 '.\\input\\H00010_Beds.csv',
 '.\\input\\H00011_Beds.csv',
 '.\\input\\H00012_Beds.csv',
 '.\\input\\H00013_Beds.csv',
 '.\\input\\H00014_Beds.csv',
 '.\\input\\H00015_Beds.csv',
 '.\\input\\H00016_Beds.csv',
 '.\\input\\H00017_Beds.csv',
 '.\\input\\H00018_Beds.csv',
 '.\\input\\H00019_Beds.csv',
 '.\\input\\H00020_Beds.csv',
 '.\\input\\H00021_Beds.csv',
 '.\\input\\H00022_Beds.csv',
 '.\\input\\H00023_Beds.csv',
 '.\\input\\H00024_Beds.csv',
 '.\\input\\H00025_Beds.csv',
 '.\\input\\H00026_Beds.csv',
 '.\\input\\H00027_Beds.csv',
 '.\\input\\H00028_Beds.csv',
 '.\\input\\H00029_Beds.csv',
 '.\\input\\H00030_Beds.csv',
 '.\\input\\H00031_Beds.csv',
 '.\\input\\H00032_Beds.csv',
 '.\\input\\H00033_Beds.csv',
 '.\\input

In [8]:
# すべての病院の空き予定リストを作成
db_empty_beds = pd.concat([pd.read_csv(file) for file in filelist]).reset_index(drop=True)[['hospital_id', 'date', '高度急性期', '急性期', '回復期', '慢性期', '介護保険施設等']].fillna(0)

In [9]:
db_empty_beds.tail(5)


,hospital_id,date,高度急性期,急性期,回復期,慢性期,介護保険施設等
3019,H00150,2021-08-17,0,14,7,8,0
3020,H00150,2021-08-18,0,14,7,8,0
3021,H00150,2021-08-19,0,14,7,8,0
3022,H00150,2021-08-20,0,14,7,8,0
3023,H00150,2021-08-21,0,14,7,8,0


In [10]:
# 日付の文字列を日付データに変換
db_empty_beds['date'] = db_empty_beds['date'].apply(lambda x: datetime.datetime.strptime(x, '%Y-%m-%d'))
db_empty_beds.head(3)

,hospital_id,date,高度急性期,急性期,回復期,慢性期,介護保険施設等
0,H00001,2021-08-01,0,0,0,11,0
1,H00001,2021-08-02,0,0,0,11,0
2,H00001,2021-08-03,0,0,0,11,0


In [11]:
min(db_empty_beds['date'])

Timestamp('2021-08-01 00:00:00')

In [12]:
#上で変換したdateの形式はTimestampsになっていて、普通に変換したものはdatetimeになっている。でも辞書のキーとして取り出しできた。
print(type(db_empty_beds['date'][0]))
print(type(datetime.datetime.strptime('2019/1/1', '%Y/%m/%d')))

<class 'pandas._libs.tslibs.timestamps.Timestamp'>
<class 'datetime.datetime'>


## 患者情報をdb_patientsに読み込み

In [13]:
# 病院の患者情報のファイルリスト作成
filepath = u'.\\input\\'
# filelist = ["H00001_Patients.csv","H00002_Patients.csv","H00003_Patients.csv"]
filelist = [p for p in os.listdir(filepath) if '_Patients.csv' in p]
filelist = [filepath + f for f in filelist]
filelist

['.\\input\\H00001_Patients.csv',
 '.\\input\\H00002_Patients.csv',
 '.\\input\\H00003_Patients.csv',
 '.\\input\\H00004_Patients.csv',
 '.\\input\\H00005_Patients.csv',
 '.\\input\\H00006_Patients.csv',
 '.\\input\\H00007_Patients.csv',
 '.\\input\\H00008_Patients.csv',
 '.\\input\\H00009_Patients.csv',
 '.\\input\\H00010_Patients.csv',
 '.\\input\\H00011_Patients.csv',
 '.\\input\\H00012_Patients.csv',
 '.\\input\\H00013_Patients.csv',
 '.\\input\\H00014_Patients.csv',
 '.\\input\\H00015_Patients.csv',
 '.\\input\\H00016_Patients.csv',
 '.\\input\\H00017_Patients.csv',
 '.\\input\\H00018_Patients.csv',
 '.\\input\\H00019_Patients.csv',
 '.\\input\\H00020_Patients.csv',
 '.\\input\\H00021_Patients.csv',
 '.\\input\\H00022_Patients.csv',
 '.\\input\\H00023_Patients.csv',
 '.\\input\\H00024_Patients.csv',
 '.\\input\\H00025_Patients.csv',
 '.\\input\\H00026_Patients.csv',
 '.\\input\\H00027_Patients.csv',
 '.\\input\\H00028_Patients.csv',
 '.\\input\\H00029_Patients.csv',
 '.\\input\\H0

In [14]:
# すべての病院の患者のリストを作成
db_patients = pd.concat([pd.read_csv(file) for file in filelist]).reset_index(drop=True)

#db_patients['expected_discharge_date'].iloc[0] = np.nan #nan値を作成
#db_patients['expected_discharge_date'].iloc[2] = np.nan #nan値を作成

#入院日をtimestamp形式に変更
db_patients['hospitalization_date'] = db_patients['hospitalization_date'].apply(lambda x: datetime.datetime.strptime(x, '%Y-%m-%d'))

#退院予定日が空欄のところがnaになっているので、入院日の120日後の日付を入れて文字列で置換。対象絞り込みの時数値で比較するため値が必要。
days_delta = 120
db_patients['expected_discharge_date'] = db_patients['expected_discharge_date'].fillna(db_patients['hospitalization_date'].apply(lambda x: (x+ datetime.timedelta(days=days_delta)).strftime('%Y/%m/%d')))

#退院予定日をtimestamp形式に変更
db_patients['expected_discharge_date'] = db_patients['expected_discharge_date'].apply(lambda x: datetime.datetime.strptime(x, '%Y-%m-%d'))

#転院先の病床機能が空欄のとき'未定'にする
db_patients['next_病床機能'] = db_patients['next_病床機能'].fillna('未定')


#next_possible_hospital_idsなどnanを空白で埋める(後でsplit(',')のときにエラーにならないように
db_patients = db_patients[['hospital_id', 'patient_id',
       'hospitalization_date', 'expected_discharge_date',
       'current_hospital_id', '割当病床機能', 'next_病床機能',
       'next_possible_hospital_ids', 'next_hospital_id']].fillna("")


In [15]:
db_patients.head(3)

,hospital_id,patient_id,hospitalization_date,expected_discharge_date,current_hospital_id,割当病床機能,next_病床機能,next_possible_hospital_ids,next_hospital_id
0,H00001,慢性期0001,2021-07-07,2021-10-01,H00001,慢性期,慢性期,,
1,H00001,慢性期0002,2021-06-07,2021-09-04,H00001,慢性期,慢性期,,
2,H00001,慢性期0003,2021-07-31,2021-10-26,H00001,慢性期,慢性期,,


In [16]:
bed_types = ['高度急性期', '急性期', '回復期', '慢性期', '介護保険施設等']

# 病院と患者のクラスを定義する

## 病院のクラス

In [17]:
class Hospital:
  """
  病院のクラス
  """
  id: str = "H99999"
  name: str = "Home"
  third_area: str = "Home"
  second_area: str = "Home"
  total_beds: int = 1
  empty_beds: dict = {datetime.datetime.strptime("2021/1/1", '%Y/%m/%d'):{"高度急性期":0, "急性期":0, "回復期":0, "慢性期":0, "介護保険施設等":0}} #list of dict(date, empty_beds) 日付別の空きベッドの数
  patients: list = [] # Patientクラスのリスト

  def __init__(self, id=id, name=name, third_area=third_area, second_area=second_area, total_beds=total_beds, empty_beds=empty_beds, patients=patients) -> None:
    self.id = id
    self.name = name
    self.third_area = third_area
    self.second_area = second_area
    self.total_beds = total_beds
    self.empty_beds = empty_beds
    self.patients = patients
    self.info = {"病院名": name, "二次医療圏": second_area}

  def get_unsettled_patients(self, date: datetime.datetime) -> list:
    """
    指定した日付以前で、転院先が決まっていない患者の情報を抽出する
    """
    unsettled_patients = [] #list of Patients()
    for p in self.patients:
      if (p.next_hospital_id == "") and (p.expected_discharge_date <= date):
        unsettled_patients.append(p)
    return unsettled_patients


  def get_unsettled_patients_by_next_bed_types(self, date: datetime.datetime, next_bed_type: Literal["高度急性期","急性期","回復期","慢性期","介護保険施設等"]):
    """
    指定した日付以前で、病床機能別の,転勤先が決まっていない患者の情報を抽出する
    """
    unsettled_patients_by_next_bed_types = {}
    unsettled_patients = []     #[Patients()]
    for p in self.patients:
      if (p.next_hospital_id == "") and (p.expected_discharge_date <= date) and (p.next_bed_type == next_bed_type):
        unsettled_patients.append(p)

    unsettled_patients_by_next_bed_types[next_bed_type] = unsettled_patients
    return unsettled_patients_by_next_bed_types


  def get_empty_beds_by_type(self, date: datetime.datetime, next_bed_type: Literal["高度急性期","急性期","回復期","慢性期","介護保険施設等"]):
    """
    指定した日付以前で、病床機能別の空き数を抽出する
    """
    empty_beds_by_type = {}
    eb = {}
    for d, b in self.empty_beds.items():
      if d <= date:
        eb[d] = b[next_bed_type]
    empty_beds_by_type[next_bed_type] = {self.id:eb}
    return empty_beds_by_type



In [18]:
d = {}
print("a" in d.keys())
a = list()
a.append("a")
print(a)

False
['a']


## 患者さんのクラス

In [19]:
class Patient:
  """
  患者さんのクラス
  """
  hospital_id: str = "H99999" #ベッドコントロールを行う対象の病院ID
  patient_id: str = "P0000000001" #患者さんの識別番号
  hospitalization_date: datetime.datetime = datetime.datetime.strptime("2021/1/1", '%Y/%m/%d') #入院日
  expected_discharge_date: datetime.datetime = datetime.datetime.strptime("2021/1/1", '%Y/%m/%d') #退院予定日
  current_hospital_id: str = "H99999" #現在の入院先の病院ID
  current_bed_type: str = "慢性期"
  next_bed_type: str = "自宅療養"
  next_possible_hospital_ids: list = [] #病院IDのリスト。転院先候補
  next_hospital_id: str = "H999999" #転院先の病院ID

  def __init__(self, hospital_id = hospital_id, patient_id = patient_id, hospitalization_date = hospitalization_date,
               expected_discharge_date = expected_discharge_date, current_hospital_id = current_hospital_id,
               current_bed_type = current_bed_type, next_bed_type = next_bed_type,
               next_possible_hospital_ids = next_possible_hospital_ids, next_hospital_id = next_hospital_id) -> None:
    self.hospital_id = hospital_id
    self.patient_id = patient_id
    self.hospitalization_date = hospitalization_date
    self.expected_discharge_date = expected_discharge_date
    self.current_hospital_id = current_hospital_id
    self.current_bed_type = current_bed_type
    self.next_bed_type = next_bed_type
    self.next_possible_hospital_ids = next_possible_hospital_ids
    self.next_hospital_id = next_hospital_id
    self.info = {"hospital_id": hospital_id, "patient_id": patient_id, "入院日": hospitalization_date, "退院(予定)日": expected_discharge_date,
                 "現在の病院": current_hospital_id, "現在の病床機能": current_bed_type, "転院先の病床機能": next_bed_type,
                 "転院先候補": next_possible_hospital_ids, "転院先":next_hospital_id}

In [20]:
# vars(Patient())

In [21]:
# ByoinA = Hospital("H00001","A","宮城県","仙台医療圏",100,{datetime.datetime.strptime("2021/6/23", '%Y/%m/%d'):{"高度急性期":0, "急性期":0, "回復期":0, "慢性期":5, "介護保険施設等":0}},[])

In [22]:
# ByoinA.patients.append(Patient("H00001","A0000000001",datetime.datetime.strptime("2021/6/14", '%Y/%m/%d'),datetime.datetime.strptime("2021/6/25", '%Y/%m/%d'), "H00001","慢性期","自宅療養", ["H00001","H00002","H99999"], "H99999"))

In [23]:
# ByoinA.patients[0].info

In [24]:
# vars(ByoinA.patients[0])

In [25]:
# vars(ByoinA)

In [26]:
# ByoinA.empty_beds[datetime.datetime.strptime("2021/6/23", '%Y/%m/%d')]

# 患者のインスタンスを作る

In [27]:
'H001'.split(",")

['H001']

In [28]:
#病院idに応じて患者情報の一覧を作成（Patient()クラスのインスタンスのリスト）
def make_patient_lists(hospital_id):
  db_patients_h = db_patients[db_patients['hospital_id'] == hospital_id]
  patients_list = db_patients_h['patient_id']
  patients = []
  for i in range(len(patients_list)):
    patient_info = db_patients_h.iloc[i]
    next_possible_hospital_ids = [] if patient_info['next_possible_hospital_ids'] == '' else patient_info['next_possible_hospital_ids'].split(",")
    p = Patient(patient_info['hospital_id'],
                          patient_info['patient_id'],
                          patient_info['hospitalization_date'],
                          patient_info['expected_discharge_date'],
                          patient_info['current_hospital_id'],
                          patient_info['割当病床機能'],
                          patient_info['next_病床機能'],
                          next_possible_hospital_ids,
                          patient_info['next_hospital_id'])
    patients.append(p)
  return patients


# 病院のインスタンスを作る

In [29]:
# 病院idを取得
hospital_ids = db_empty_beds['hospital_id'].unique()
hospital_ids

array(['H00001', 'H00002', 'H00003', 'H00004', 'H00005', 'H00006',
       'H00007', 'H00008', 'H00009', 'H00010', 'H00011', 'H00012',
       'H00013', 'H00014', 'H00015', 'H00016', 'H00017', 'H00018',
       'H00019', 'H00020', 'H00021', 'H00022', 'H00023', 'H00024',
       'H00025', 'H00026', 'H00027', 'H00028', 'H00029', 'H00030',
       'H00031', 'H00032', 'H00033', 'H00034', 'H00035', 'H00036',
       'H00037', 'H00038', 'H00039', 'H00040', 'H00041', 'H00042',
       'H00043', 'H00044', 'H00045', 'H00046', 'H00047', 'H00048',
       'H00049', 'H00050', 'H00051', 'H00052', 'H00053', 'H00054',
       'H00055', 'H00056', 'H00057', 'H00058', 'H00059', 'H00060',
       'H00061', 'H00062', 'H00063', 'H00064', 'H00065', 'H00066',
       'H00067', 'H00068', 'H00069', 'H00070', 'H00071', 'H00072',
       'H00073', 'H00074', 'H00075', 'H00076', 'H00077', 'H00078',
       'H00079', 'H00080', 'H00081', 'H00082', 'H00083', 'H00084',
       'H00085', 'H00086', 'H00087', 'H00088', 'H00089', 'H000

In [30]:
#病院idごとにHospital()クラスのインスタンスを作成
hospitals = {}
for h in hospital_ids:

  #病院別に日付をキーにした空き病床辞書を作成
  empty_beds = db_empty_beds[db_empty_beds['hospital_id'] == h][db_empty_beds.columns[1:]].set_index('date')
  empty_beds = empty_beds.to_dict('index')

  #idをもとにDB_Hospitalsから情報を取得
  hospital_info = DB_Hospitals.loc[h]

  #病院の患者情報を取得
  patients = make_patient_lists(h)

  hospitals[h] = Hospital(h, hospital_info['医療機関名称'], hospital_info['三次医療圏'], hospital_info['二次医療圏'], hospital_info['全体'], empty_beds, patients)


In [31]:
hospitals

{'H00001': <__main__.Hospital at 0x1a41988adb0>,
 'H00002': <__main__.Hospital at 0x1a41fbd3ef0>,
 'H00003': <__main__.Hospital at 0x1a420ab3cb0>,
 'H00004': <__main__.Hospital at 0x1a41fd89310>,
 'H00005': <__main__.Hospital at 0x1a420adc350>,
 'H00006': <__main__.Hospital at 0x1a420b54680>,
 'H00007': <__main__.Hospital at 0x1a420a80e90>,
 'H00008': <__main__.Hospital at 0x1a420afe4e0>,
 'H00009': <__main__.Hospital at 0x1a421c95a60>,
 'H00010': <__main__.Hospital at 0x1a4221b78c0>,
 'H00011': <__main__.Hospital at 0x1a421cef830>,
 'H00012': <__main__.Hospital at 0x1a420ab1b50>,
 'H00013': <__main__.Hospital at 0x1a421c97500>,
 'H00014': <__main__.Hospital at 0x1a4221b44a0>,
 'H00015': <__main__.Hospital at 0x1a421d399a0>,
 'H00016': <__main__.Hospital at 0x1a420add5e0>,
 'H00017': <__main__.Hospital at 0x1a420b54740>,
 'H00018': <__main__.Hospital at 0x1a42214ef90>,
 'H00019': <__main__.Hospital at 0x1a422182ba0>,
 'H00020': <__main__.Hospital at 0x1a41988aba0>,
 'H00021': <__main__

In [32]:
print(hospitals['H00001'].name)
hospitals['H00001'].patients[10].info


貝山中央病院


{'hospital_id': 'H00001',
 'patient_id': '慢性期0011',
 '入院日': Timestamp('2021-05-26 00:00:00'),
 '退院(予定)日': Timestamp('2021-08-16 00:00:00'),
 '現在の病院': 'H00001',
 '現在の病床機能': '慢性期',
 '転院先の病床機能': '慢性期',
 '転院先候補': [],
 '転院先': ''}

In [33]:
ls = hospitals['H00001'].get_unsettled_patients(datetime.datetime.strptime('2021/8/10', '%Y/%m/%d'))

In [34]:
for l in ls:
  print(l.info)
#  print(l.expected_discharge_date <= datetime.datetime.strptime('2021/7/10', '%Y/%m/%d')


{'hospital_id': 'H00001', 'patient_id': '慢性期0008', '入院日': Timestamp('2021-06-09 00:00:00'), '退院(予定)日': Timestamp('2021-08-10 00:00:00'), '現在の病院': 'H00001', '現在の病床機能': '慢性期', '転院先の病床機能': '慢性期', '転院先候補': [], '転院先': ''}
{'hospital_id': 'H00001', 'patient_id': '慢性期0010', '入院日': Timestamp('2021-06-18 00:00:00'), '退院(予定)日': Timestamp('2021-08-10 00:00:00'), '現在の病院': 'H00001', '現在の病床機能': '慢性期', '転院先の病床機能': '慢性期', '転院先候補': [], '転院先': ''}
{'hospital_id': 'H00001', 'patient_id': '慢性期0027', '入院日': Timestamp('2021-05-11 00:00:00'), '退院(予定)日': Timestamp('2021-08-07 00:00:00'), '現在の病院': 'H00001', '現在の病床機能': '慢性期', '転院先の病床機能': '慢性期', '転院先候補': [], '転院先': ''}


# 患者iごとの次の転院先hを求める
①まずは最適化対象の患者を抽出し  
②転院先の病床タイプごとに最適化計算を行う

## 最適化期間を決める

In [35]:
# 何日までのスケジュールで最適化するかを決める
# この条件で患者を抽出した後、予定が未定の最初の日をfirst_dateとして、その日からtarget_dateの間の日数（同日なら1日と扱う）を範囲Dとして最適化
target_date = "2021/8/10"
target_date = datetime.datetime.strptime(target_date, '%Y/%m/%d')
today = min(db_empty_beds['date'])
D0 = target_date - today
D0 = D0.days + 1
print('最適化対象期間: ', today.date(), 'to', target_date.date())
print(D0,'日')


最適化対象期間:  2021-08-01 to 2021-08-10
10 日


## 最適化対象期間内で転院することが決まっていて、かつ転院先が決まっていない患者の情報を抽出


In [36]:
#最適化対象期間内で転院することが決まっていて、かつ転院先が決まっていない患者の情報を抽出
#後で、unsettled_patients_by_next_bed_typesに、転院先の病床タイプごとに分類（病床タイプごとに最適化する）
unsettled_patients = []
for h in hospitals.values():
  unsettled_patients.extend(h.get_unsettled_patients(target_date))
print("対象期間：", target_date, "、対象患者数:",len(unsettled_patients))

対象期間： 2021-08-10 00:00:00 、対象患者数: 2761


In [37]:
for i in unsettled_patients:
  print(i.next_bed_type)

慢性期
慢性期
慢性期
慢性期
慢性期
慢性期
慢性期
回復期
回復期
回復期
回復期
回復期
回復期
回復期
回復期
回復期
回復期
回復期
回復期
回復期
回復期
回復期
回復期
回復期
回復期
回復期
回復期
回復期
回復期
回復期
回復期
回復期
回復期
回復期
回復期
回復期
回復期
回復期
回復期
回復期
回復期
回復期
回復期
回復期
回復期
回復期
回復期
回復期
回復期
回復期
回復期
回復期
回復期
回復期
回復期
回復期
回復期
回復期
回復期
回復期
回復期
回復期
回復期
回復期
回復期
回復期
回復期
回復期
回復期
回復期
回復期
回復期
回復期
回復期
回復期
回復期
回復期
回復期
回復期
回復期
回復期
回復期
回復期
回復期
回復期
回復期
回復期
回復期
回復期
回復期
回復期
回復期
回復期
回復期
回復期
回復期
回復期
回復期
回復期
回復期
回復期
回復期
回復期
回復期
回復期
回復期
回復期
回復期
回復期
回復期
回復期
回復期
回復期
回復期
回復期
回復期
回復期
回復期
回復期
回復期
回復期
回復期
回復期
回復期
回復期
回復期
回復期
回復期
回復期
回復期
回復期
回復期
回復期
回復期
回復期
回復期
回復期
回復期
回復期
回復期
回復期
回復期
回復期
回復期
回復期
回復期
回復期
回復期
回復期
回復期
回復期
回復期
回復期
回復期
回復期
回復期
回復期
回復期
回復期
回復期
回復期
回復期
回復期
回復期
回復期
回復期
回復期
回復期
回復期
回復期
回復期
回復期
回復期
回復期
回復期
回復期
回復期
回復期
回復期
回復期
回復期
回復期
回復期
回復期
回復期
回復期
回復期
回復期
回復期
回復期
回復期
回復期
回復期
回復期
回復期
回復期
回復期
回復期
回復期
回復期
回復期
回復期
回復期
回復期
回復期
回復期
回復期
回復期
回復期
回復期
回復期
回復期
回復期
回復期
回復期
回復期
回復期
回復期
回復期
回復期
回復期
回復期
回復期
回復期
回復期
回復期
回復期
回復期
回復期
回復期
回復期
回復期
回復期
回復期
回復期
回復期
回復期
回復期
回復期
回復期
回復期
回復期
回復期
回復期
回復期
回復期
回復期
回復期
回復期
回復期


## 転院先の病床タイプごとに患者の情報を分類

In [38]:
#転院先の病床タイプごとに患者の情報を分類
unsettled_patients_by_next_bed_types = {'高度急性期':[], '急性期':[], '回復期':[], '慢性期':[], '介護保険施設等':[], '未定':[]}
for h in hospitals.values():
  for bed_type in bed_types+['未定']:
    unsettled_patients_by_next_bed_types[bed_type].extend(h.get_unsettled_patients_by_next_bed_types(target_date, bed_type)[bed_type])


In [39]:
#確認
#unsettled_patients_by_next_bed_types
'''
{'高度急性期': [],
 '急性期': [],
 '回復期': [<__main__.Patient at 0x170baeffdc8>,
  <__main__.Patient at 0x170baeff908>,
  <__main__.Patient at 0x170bae8f488>,
  <__main__.Patient at 0x170bae8fa08>,
  <__main__.Patient at 0x170bb2ae888>,
  <__main__.Patient at 0x170bb2ae748>,
  <__main__.Patient at 0x170bb2abf48>,
  <__main__.Patient at 0x170bb2abc08>,
    ...],
 '慢性期': [<__main__.Patient at 0x170bb2ae688>,
  <__main__.Patient at 0x170baea51c8>,
  <__main__.Patient at 0x170bb2bd508>,
  <__main__.Patient at 0x170bae66108>,
  <__main__.Patient at 0x170bb2c6408>,
    <__main__.Patient at 0x170bc1b27c8>],
 '介護保険施設等': [],
 '未定': []}
'''

"\n{'高度急性期': [],\n '急性期': [],\n '回復期': [<__main__.Patient at 0x170baeffdc8>,\n  <__main__.Patient at 0x170baeff908>,\n  <__main__.Patient at 0x170bae8f488>,\n  <__main__.Patient at 0x170bae8fa08>,\n  <__main__.Patient at 0x170bb2ae888>,\n  <__main__.Patient at 0x170bb2ae748>,\n  <__main__.Patient at 0x170bb2abf48>,\n  <__main__.Patient at 0x170bb2abc08>,\n    ...],\n '慢性期': [<__main__.Patient at 0x170bb2ae688>,\n  <__main__.Patient at 0x170baea51c8>,\n  <__main__.Patient at 0x170bb2bd508>,\n  <__main__.Patient at 0x170bae66108>,\n  <__main__.Patient at 0x170bb2c6408>,\n    <__main__.Patient at 0x170bc1b27c8>],\n '介護保険施設等': [],\n '未定': []}\n"

## 病床タイプ別に、病院ごとの空きベッド情報を、対象期間の日別で抽出

In [40]:
#病床タイプ別に、病院ごとの空きベッド情報を、対象期間の日別で抽出
empty_beds_by_types = {'高度急性期':{}, '急性期':{}, '回復期':{}, '慢性期':{}, '介護保険施設等':{}}
for h in hospitals.values():
  for bed_type in bed_types:
    empty_beds_by_types[bed_type].update(h.get_empty_beds_by_type(target_date, bed_type)[bed_type])


In [41]:
empty_beds_by_types

{'高度急性期': {'H00001': {Timestamp('2021-08-01 00:00:00'): 0,
   Timestamp('2021-08-02 00:00:00'): 0,
   Timestamp('2021-08-03 00:00:00'): 0,
   Timestamp('2021-08-04 00:00:00'): 0,
   Timestamp('2021-08-05 00:00:00'): 0,
   Timestamp('2021-08-06 00:00:00'): 0,
   Timestamp('2021-08-07 00:00:00'): 0,
   Timestamp('2021-08-08 00:00:00'): 0,
   Timestamp('2021-08-09 00:00:00'): 0,
   Timestamp('2021-08-10 00:00:00'): 0},
  'H00002': {Timestamp('2021-08-01 00:00:00'): 0,
   Timestamp('2021-08-02 00:00:00'): 0,
   Timestamp('2021-08-03 00:00:00'): 0,
   Timestamp('2021-08-04 00:00:00'): 0,
   Timestamp('2021-08-05 00:00:00'): 0,
   Timestamp('2021-08-06 00:00:00'): 0,
   Timestamp('2021-08-07 00:00:00'): 0,
   Timestamp('2021-08-08 00:00:00'): 0,
   Timestamp('2021-08-09 00:00:00'): 0,
   Timestamp('2021-08-10 00:00:00'): 0},
  'H00003': {Timestamp('2021-08-01 00:00:00'): 87,
   Timestamp('2021-08-02 00:00:00'): 87,
   Timestamp('2021-08-03 00:00:00'): 87,
   Timestamp('2021-08-04 00:00:00'):

## 転院先が「慢性期」の患者について転院先を最適化

In [42]:
# 病床タイプを決めて、それに応じて転院先が未定の患者情報を取得
bed_type = '慢性期'
unsettled_patients_by_next_bed_type = unsettled_patients_by_next_bed_types[bed_type]


In [43]:
# 病床タイプが空いている病院を絞り込み
hospitals_by_empty_bed_type = []
for hospital_id,date in empty_beds_by_types[bed_type].items():
  # print(hospital_id,date)
  bed = 0
  for bed in date.values():
    bed = bed + 1 if bed > 0 else bed
  if bed >0:
    hospitals_by_empty_bed_type.append(hospital_id)


print(len(hospitals_by_empty_bed_type))
print(hospitals_by_empty_bed_type)


62
['H00001', 'H00002', 'H00007', 'H00013', 'H00015', 'H00018', 'H00020', 'H00023', 'H00027', 'H00031', 'H00032', 'H00035', 'H00036', 'H00039', 'H00043', 'H00046', 'H00047', 'H00049', 'H00050', 'H00051', 'H00052', 'H00055', 'H00057', 'H00058', 'H00059', 'H00060', 'H00063', 'H00064', 'H00066', 'H00067', 'H00069', 'H00073', 'H00083', 'H00084', 'H00085', 'H00086', 'H00087', 'H00088', 'H00090', 'H00093', 'H00094', 'H00095', 'H00098', 'H00099', 'H00100', 'H00101', 'H00103', 'H00109', 'H00118', 'H00121', 'H00123', 'H00124', 'H00125', 'H00127', 'H00132', 'H00133', 'H00134', 'H00136', 'H00138', 'H00140', 'H00145', 'H00150']


In [44]:
# 最適化対象患者さんの入院中の病院リスト
hospital_ids_of_patients_by_next_bed_type = []
for p in unsettled_patients_by_next_bed_type:
  hospital_ids_of_patients_by_next_bed_type.append(p.hospital_id)
hospital_ids_of_patients_by_next_bed_type = set(hospital_ids_of_patients_by_next_bed_type)
print(hospital_ids_of_patients_by_next_bed_type)

{'H00001', 'H00093', 'H00058', 'H00085', 'H00098', 'H00057', 'H00020', 'H00073', 'H00095', 'H00150', 'H00086', 'H00007', 'H00124', 'H00123', 'H00002', 'H00050', 'H00090', 'H00100', 'H00013', 'H00036', 'H00060', 'H00121', 'H00127', 'H00133', 'H00138', 'H00018', 'H00101', 'H00032', 'H00083', 'H00047', 'H00039', 'H00043', 'H00015', 'H00069', 'H00051', 'H00063', 'H00066', 'H00046', 'H00059', 'H00145', 'H00027', 'H00035', 'H00088', 'H00099', 'H00132', 'H00136', 'H00084', 'H00087', 'H00064', 'H00103', 'H00052', 'H00067'}


### 定数定義

In [45]:
# 最適化対象患者さんの最短の転院日
first_date = target_date
for p in unsettled_patients_by_next_bed_type:
  first_date = p.expected_discharge_date if first_date > p.expected_discharge_date else first_date
print(first_date)

2021-08-06 00:00:00


In [46]:
# 定数定義
N = len(unsettled_patients_by_next_bed_type)
print(bed_type,"の病床に移る必要のある患者数 N",N)
H0 = len(hospital_ids_of_patients_by_next_bed_type) #患者が現在入院している病院の数
print("患者が現在入院している病院の数",H0,hospital_ids_of_patients_by_next_bed_type)
H = len(hospitals_by_empty_bed_type) #転院先の数
print("転院先の数 H",H,hospitals_by_empty_bed_type)
print('最適化期間',today,'to',target_date)
print('最適化開始日(転院先が決まっていない患者の中での最初の転院日)：first_date',first_date)
D = (target_date - first_date).days + 1
print('最適化対象日数 D',D)

慢性期 の病床に移る必要のある患者数 N 165
患者が現在入院している病院の数 52 {'H00001', 'H00093', 'H00058', 'H00085', 'H00098', 'H00057', 'H00020', 'H00073', 'H00095', 'H00150', 'H00086', 'H00007', 'H00124', 'H00123', 'H00002', 'H00050', 'H00090', 'H00100', 'H00013', 'H00036', 'H00060', 'H00121', 'H00127', 'H00133', 'H00138', 'H00018', 'H00101', 'H00032', 'H00083', 'H00047', 'H00039', 'H00043', 'H00015', 'H00069', 'H00051', 'H00063', 'H00066', 'H00046', 'H00059', 'H00145', 'H00027', 'H00035', 'H00088', 'H00099', 'H00132', 'H00136', 'H00084', 'H00087', 'H00064', 'H00103', 'H00052', 'H00067'}
転院先の数 H 62 ['H00001', 'H00002', 'H00007', 'H00013', 'H00015', 'H00018', 'H00020', 'H00023', 'H00027', 'H00031', 'H00032', 'H00035', 'H00036', 'H00039', 'H00043', 'H00046', 'H00047', 'H00049', 'H00050', 'H00051', 'H00052', 'H00055', 'H00057', 'H00058', 'H00059', 'H00060', 'H00063', 'H00064', 'H00066', 'H00067', 'H00069', 'H00073', 'H00083', 'H00084', 'H00085', 'H00086', 'H00087', 'H00088', 'H00090', 'H00093', 'H00094', 'H00095', 'H0


$$
H_{total} = α\sum^N_{i=1} \sum^H_{h=1} F_{i,h}x_{i, h} 
+ β \sum^H_{h=1} \sum^D_{d=1} \sum^N_{i=1}\left( -D_{i,d}b_{h,d}x_{i,h}\right)
+ λ \sum^N_{i=1}
\left(1-\sum^H_{h=1} x_{i,h}\right)^2
$$

$$b_{h,d} = B_{h,d} - \sum^d_{t=1}\sum^N_{j=1}D_{j,t}x_{j,h}$$
$$x_{i,h}\in \left\{ 0,1 \right\} : 患者iの転院先が病院hのとき1$$
$$F_{i,h} : 患者iが病院hに転院するときの望ましさ$$
$$D_{i,d}\in \left\{ 0,1 \right\} : 患者iの退院日（転院日）がd日目のとき1$$
$$b_{h,d} : 転院受け入れによる変化を含む、病院hのd日目の空きベッド数$$
$$B_{h,d} : 現時点で予めわかっている病院hのd日目における空きベッド数$$

$N$は最適化対象の患者数（dD日後までに退院日を迎えるが、転院先が決まっていない人）  
$H$は転院先候補の病院数  
$D$は最適化対象の期間（今日からD日目（D-1 日後）までを対象）  


第一項の$F_{i,h}$の要素は患者iの現在の入院先$h_i$に対し$h=h_1$となるとき、転院先が同じにならないように高いコストを設定。  
その他は、同じ病院内で病床が変わるとき（急性期→回復期）は低コストとか、病院の地域が同じであれば低コストとかにしている。  
$F_{i,h}$の要素はすべて正の値としていて、$x_{i,h}$が0になる方向のコスト関数。  
  
  
第二項は、患者$i$の転院日$d$に$b_{h,d}$が大きい病院を選ぶことでコストが下がるように意図している。  
特定の病院に転院先が集中すると空きベッド$b_{h,d}$がマイナスになり、コスト上昇するので、転院数は病床数を超えないという制約条件の代わりにしている（つもり。うまく行かないので精査中）  
ここで、$b_{h,d}$は基本的にプラスの値なので、マイナスをつけている第二項は、$x_{i,h}$が1になる方向のコスト関数（ただし全部1になると病床が足りなくなってコストがプラスになることもある）  
  
__現在の課題としては、$x_{i,h}$に対する第一項、第二項の影響が逆なので、制約条件の第三項が守られるような係数A,B,Cの比率設定が難しい。__  

__また、問題のスケールを変えると係数も変わるので、スケールに応じて適切な係数を計算したい。__

今の所、$x_{i,h}$にかかる係数の和を$N*H$で割った値を$x_{i,h}$が1増えるときのコストの期待値として、それぞれの期待値の逆数を係数とした。  
第1項については、$x_{i,h}$のうちN個が1になるまでは、罰金項のコストのほうが高くなる。  
第2項については、$x_{i,h}$のうちN個が1を超えると、罰金項のコストのほうが高くなる。  
上記のように、罰金項の係数が1のとき、第1項、第2項のコストの期待値とそれぞれつり合いが取れるので、第三項の制約条件の係数λをコスト項の数（2）とすることで全体とつり合いがとれるイメージ。  
つり合い状態では罰金項が守られないので、10％増くらいで設定（α=1, β=1, λ=2.2)がよい?)。  

第2項は$x_{i,h}$の値によって係数が変わるので、すべての$x_{i,h}$を1で計算したが、もっと精緻にやるべき。  
第2項がアニーリングの使い所なのでちゃんと影響が出るように、いい感じにしたい。

### 変数xとその制約条件

In [47]:
lam1 = 1
'''
# 数式通り計算・・・遅い、対角化できていないので最後にハミルトニアンを出すときに時間がかかる
QUBO_constr1 = {}
for i in range(N):
  for h in range(H):
    for k in range(H):

      QUBO_constr1[(i*H+k,i*H+h)] = lam1
      if h == k:
        QUBO_constr1[(i*H+h,i*H+k)] = QUBO_constr1[(i*H+h,i*H+k)] - 2*lam1
'''
# 対角成分は上三角に寄せる形で計算
QUBO_constr1 = {}
for i in range(N):
  for h in range(H):
    for k in range(h+1):
      if h != k:
        QUBO_constr1[(i*H+k,i*H+h)] = 2*lam1
      if h == k:
        QUBO_constr1[(i*H+k,i*H+h)] = - lam1

In [48]:
sum(QUBO_constr1.values())

613800

In [49]:
len(QUBO_constr1.keys())

322245

### コスト関数1
望ましい転院先のパラメータをもとに荷重をつける

#### Fih:患者iにとっての転院先hの望ましさ（自宅との距離が近いとか）



In [50]:
#Fih:患者iにとっての転院先hの望ましさ（自宅との距離が近いとか）
Fih = np.zeros(N*H).reshape(N,H)
for i in range(N):
  for h in range(H):
    # 今の病床タイプと、転院先の病床タイプが同じ場合は、今の病院を選ばないようにする。
    # 今の病床タイプと、転院先の病床タイプが異なる場合は、今の病院に病床があるなら積極的に選ぶ
    if unsettled_patients_by_next_bed_type[i].hospital_id == hospitals_by_empty_bed_type[h]:
      if unsettled_patients_by_next_bed_type[i].current_bed_type == unsettled_patients_by_next_bed_type[i].next_bed_type:
        Fih[i,h] = 100 #今の病院を転院先に選ばない
      else:
        Fih[i,h] = 1 #病床タイプが異なるなら、同じ病院内で転院　最優先を1として、これを基準にほかが何倍選ばれにくいかを決めていく。


    # "二次医療圏"がおなじ病院を優先
    elif hospitals[unsettled_patients_by_next_bed_type[i].hospital_id].second_area == hospitals[hospitals_by_empty_bed_type[h]].second_area:
      Fih[i,h] = 10

    # 特に好ましいパラメータがない場合
    else:
      Fih[i,h] = 50


In [51]:
print(hospitals[unsettled_patients_by_next_bed_type[00].hospital_id].second_area)
print(hospitals[list(hospitals.keys())[0]].second_area)


仙台医療圏
仙台医療圏


#### 望ましい転院先のハミルトニアン

In [52]:
QUBO_cost1 = {}
for i in range(N):
  for h in range(H):
    QUBO_cost1[i*H+h,i*H+h] = Fih[i,h]

In [53]:
cost1_sum = sum(QUBO_cost1.values())
alpha = H*N/cost1_sum
print('QUBO_cost1の値の合計',cost1_sum)
print('QUBO_cost1の係数α',alpha)

QUBO_cost1の値の合計 373030.0
QUBO_cost1の係数α 0.02742406776934831


### コスト関数２
転院予定日に空き予定の多い病院に入る

#### 患者iの転院日がd日後のとき1の行列Didを定義

In [54]:
first_date

Timestamp('2021-08-06 00:00:00')

In [55]:
#患者iの転院日がfirst_date(対象患者全体での最短の退院日）のd日後のとき1の行列Didを定義
Did = np.zeros(N*D, dtype='int8').reshape(N,D)
for i in range(N):
  for d in range(D):
    if unsettled_patients_by_next_bed_type[i].expected_discharge_date == first_date + datetime.timedelta(days=d):
      Did[i,d] = 1
  print(unsettled_patients_by_next_bed_type[i].hospital_id, unsettled_patients_by_next_bed_type[i].patient_id, Did[i,:])
#print(Did)
Did.astype('int8')

H00001 慢性期0008 [0 0 0 0 1]
H00001 慢性期0010 [0 0 0 0 1]
H00001 慢性期0027 [0 1 0 0 0]
H00002 慢性期0010 [0 0 0 0 1]
H00002 慢性期0011 [0 0 0 0 1]
H00002 慢性期0027 [0 0 1 0 0]
H00002 慢性期0033 [0 1 0 0 0]
H00007 慢性期0006 [0 0 0 1 0]
H00007 慢性期0046 [0 0 0 1 0]
H00007 慢性期0069 [0 0 0 1 0]
H00007 慢性期0082 [0 1 0 0 0]
H00013 慢性期0033 [0 0 0 0 1]
H00015 慢性期0033 [0 1 0 0 0]
H00015 慢性期0035 [0 1 0 0 0]
H00015 慢性期0067 [0 1 0 0 0]
H00015 慢性期0072 [0 0 0 0 1]
H00015 慢性期0082 [0 0 0 1 0]
H00015 慢性期0099 [0 0 0 1 0]
H00015 慢性期0107 [1 0 0 0 0]
H00015 慢性期0128 [0 1 0 0 0]
H00015 慢性期0140 [1 0 0 0 0]
H00015 慢性期0172 [0 0 0 0 1]
H00015 慢性期0180 [0 1 0 0 0]
H00018 慢性期0008 [0 0 1 0 0]
H00020 慢性期0001 [0 0 1 0 0]
H00020 慢性期0015 [0 0 1 0 0]
H00020 慢性期0028 [0 1 0 0 0]
H00027 慢性期0003 [1 0 0 0 0]
H00027 慢性期0013 [0 0 0 1 0]
H00032 慢性期0003 [0 0 0 1 0]
H00035 慢性期0012 [0 0 0 1 0]
H00035 慢性期0019 [0 1 0 0 0]
H00035 慢性期0026 [0 0 0 1 0]
H00035 慢性期0073 [1 0 0 0 0]
H00035 慢性期0122 [0 1 0 0 0]
H00035 慢性期0141 [0 0 0 1 0]
H00035 慢性期0151 [0 1 0 0 0]
H

array([[0, 0, 0, 0, 1],
       [0, 0, 0, 0, 1],
       [0, 1, 0, 0, 0],
       [0, 0, 0, 0, 1],
       [0, 0, 0, 0, 1],
       [0, 0, 1, 0, 0],
       [0, 1, 0, 0, 0],
       [0, 0, 0, 1, 0],
       [0, 0, 0, 1, 0],
       [0, 0, 0, 1, 0],
       [0, 1, 0, 0, 0],
       [0, 0, 0, 0, 1],
       [0, 1, 0, 0, 0],
       [0, 1, 0, 0, 0],
       [0, 1, 0, 0, 0],
       [0, 0, 0, 0, 1],
       [0, 0, 0, 1, 0],
       [0, 0, 0, 1, 0],
       [1, 0, 0, 0, 0],
       [0, 1, 0, 0, 0],
       [1, 0, 0, 0, 0],
       [0, 0, 0, 0, 1],
       [0, 1, 0, 0, 0],
       [0, 0, 1, 0, 0],
       [0, 0, 1, 0, 0],
       [0, 0, 1, 0, 0],
       [0, 1, 0, 0, 0],
       [1, 0, 0, 0, 0],
       [0, 0, 0, 1, 0],
       [0, 0, 0, 1, 0],
       [0, 0, 0, 1, 0],
       [0, 1, 0, 0, 0],
       [0, 0, 0, 1, 0],
       [1, 0, 0, 0, 0],
       [0, 1, 0, 0, 0],
       [0, 0, 0, 1, 0],
       [0, 1, 0, 0, 0],
       [0, 0, 0, 1, 0],
       [0, 1, 0, 0, 0],
       [1, 0, 0, 0, 0],
       [0, 0, 0, 0, 1],
       [0, 0, 0,

In [56]:
hospitals['H00001'].get_empty_beds_by_type(today+datetime.timedelta(days=3),bed_type)

{'慢性期': {'H00001': {Timestamp('2021-08-01 00:00:00'): 11,
   Timestamp('2021-08-02 00:00:00'): 11,
   Timestamp('2021-08-03 00:00:00'): 11,
   Timestamp('2021-08-04 00:00:00'): 11}}}

#### 現時点で予めわかっている病院hのd日後の空きベット数B_h,d

In [57]:
# 現時点で予めわかっている病院hのfirst_dateからd日後の空きベット数
Bhd = np.zeros(H*D, dtype='int8').reshape(H,D)
empty_beds = empty_beds_by_types[bed_type]
for h,hospital_id in enumerate(hospitals_by_empty_bed_type):
  for d in range(D):
    date = first_date + datetime.timedelta(days=d)
    Bhd[h,d] = empty_beds[hospital_id][date]
  print(hospital_id,Bhd[h,:])
Bhd.astype('int8')

H00001 [11 11 11 11 11]
H00002 [2 2 2 2 2]
H00007 [4 4 4 4 4]
H00013 [9 9 9 9 9]
H00015 [36 36 36 36 36]
H00018 [9 9 9 9 9]
H00020 [10 10 10 10 10]
H00023 [3 3 3 3 3]
H00027 [16 16 16 16 16]
H00031 [6 6 6 6 6]
H00032 [10 10 10 10 10]
H00035 [11 11 11 11 11]
H00036 [12 12 12 12 12]
H00039 [37 37 37 37 37]
H00043 [8 8 8 8 8]
H00046 [17 17 17 17 17]
H00047 [9 9 9 9 9]
H00049 [12 12 12 12 12]
H00050 [1 1 1 1 1]
H00051 [6 6 6 6 6]
H00052 [10 10 10 10 10]
H00055 [10 10 10 10 10]
H00057 [28 28 28 28 28]
H00058 [15 15 15 15 15]
H00059 [2 2 2 2 2]
H00060 [2 2 2 2 2]
H00063 [6 6 6 6 6]
H00064 [26 26 26 26 26]
H00066 [1 1 1 1 1]
H00067 [7 7 7 7 7]
H00069 [9 9 9 9 9]
H00073 [7 7 7 7 7]
H00083 [3 3 3 3 3]
H00084 [11 11 11 11 11]
H00085 [3 3 3 3 3]
H00086 [13 13 13 13 13]
H00087 [2 2 2 2 2]
H00088 [3 3 3 3 3]
H00090 [5 5 5 5 5]
H00093 [7 7 7 7 7]
H00094 [2 2 2 2 2]
H00095 [9 9 9 9 9]
H00098 [3 3 3 3 3]
H00099 [6 6 6 6 6]
H00100 [7 7 7 7 7]
H00101 [5 5 5 5 5]
H00103 [30 30 30 30 30]
H00109 [1 1 1 1 1

array([[11, 11, 11, 11, 11],
       [ 2,  2,  2,  2,  2],
       [ 4,  4,  4,  4,  4],
       [ 9,  9,  9,  9,  9],
       [36, 36, 36, 36, 36],
       [ 9,  9,  9,  9,  9],
       [10, 10, 10, 10, 10],
       [ 3,  3,  3,  3,  3],
       [16, 16, 16, 16, 16],
       [ 6,  6,  6,  6,  6],
       [10, 10, 10, 10, 10],
       [11, 11, 11, 11, 11],
       [12, 12, 12, 12, 12],
       [37, 37, 37, 37, 37],
       [ 8,  8,  8,  8,  8],
       [17, 17, 17, 17, 17],
       [ 9,  9,  9,  9,  9],
       [12, 12, 12, 12, 12],
       [ 1,  1,  1,  1,  1],
       [ 6,  6,  6,  6,  6],
       [10, 10, 10, 10, 10],
       [10, 10, 10, 10, 10],
       [28, 28, 28, 28, 28],
       [15, 15, 15, 15, 15],
       [ 2,  2,  2,  2,  2],
       [ 2,  2,  2,  2,  2],
       [ 6,  6,  6,  6,  6],
       [26, 26, 26, 26, 26],
       [ 1,  1,  1,  1,  1],
       [ 7,  7,  7,  7,  7],
       [ 9,  9,  9,  9,  9],
       [ 7,  7,  7,  7,  7],
       [ 3,  3,  3,  3,  3],
       [11, 11, 11, 11, 11],
       [ 3,  3

#### 全期間を通して空き病床が多くなるような転院計画(空き病床がマイナスになるとコスト増、空き病床の多い病院に転院するときコスト減）

1項目の1つ目の要素

In [58]:
matE = np.tile(np.identity(H, dtype='int8'), (N,1))
matE.shape #(H*N,H)

(10230, 62)

In [59]:
matH11 = matE@Bhd

1項目2つ目の要素H12

In [60]:
# for文を使わずに行ごとにリピートして縦に並べるやり方
# https://tawara.hatenablog.com/entry/2020/05/03/223519
matE2 = np.tile(np.identity(N, dtype='int8')[:,None],(1,H,1))
matE2.shape



(165, 62, 165)

In [61]:
matE2.ravel().reshape(H*N,N)

array([[1, 0, 0, ..., 0, 0, 0],
       [1, 0, 0, ..., 0, 0, 0],
       [1, 0, 0, ..., 0, 0, 0],
       ...,
       [0, 0, 0, ..., 0, 0, 1],
       [0, 0, 0, ..., 0, 0, 1],
       [0, 0, 0, ..., 0, 0, 1]], dtype=int8)

In [62]:
matH12 = matE2.ravel().reshape(H*N,N)@Did

In [63]:
matH12.shape

(10230, 5)

H11とH12のアダマール積を取って、各行の和をとる（＝xihの係数）

In [64]:
matH1_ = -matH11*matH12@(np.array([1]*D, dtype='int8').reshape(-1,1))

In [65]:
matH1 = np.diag(matH1_.ravel())
print(matH1.shape)

(10230, 10230)


2項目の要素  
=Did@(下三角行列)@Did.T

In [66]:
matH2_ = Did@np.tri(D, dtype='int8')@Did.T

In [67]:
matH2_.shape

(165, 165)

N,N 行列をXih,Xjhに対応する係数のマトリックス、N*H,N*Hに変換  
tileを使って一回で延長しようとするとメモリエラーがでた。floatだとメモリオーバーになるが、int8にすると行ける

In [68]:
tmpH2 = matH2_
# tmpH2_0 = np.tile(tmpH2[:,None,:,None],(1,H,1,H)).ravel().reshape(H*N,H*N)
tmpH2_0 = np.tile(tmpH2[:,None,:,None],(1,H,1,H)).reshape(H*N,H*N)

In [69]:
#H×Hの行列を縦横にN個並べた行列をかけてmatH2を作成
matH2 = tmpH2_0*np.tile(np.identity(H, dtype='int8'),(N,N))

In [70]:
matH = matH1 + matH2

In [71]:
matH1

array([[-11,   0,   0, ...,   0,   0,   0],
       [  0,  -2,   0, ...,   0,   0,   0],
       [  0,   0,  -4, ...,   0,   0,   0],
       ...,
       [  0,   0,   0, ...,  -6,   0,   0],
       [  0,   0,   0, ...,   0,  -6,   0],
       [  0,   0,   0, ...,   0,   0,  -8]], dtype=int8)

In [72]:
matH2

array([[1, 0, 0, ..., 0, 0, 0],
       [0, 1, 0, ..., 0, 0, 0],
       [0, 0, 1, ..., 0, 0, 0],
       ...,
       [0, 0, 0, ..., 1, 0, 0],
       [0, 0, 0, ..., 0, 1, 0],
       [0, 0, 0, ..., 0, 0, 1]], dtype=int8)

In [73]:
matH

array([[-10,   0,   0, ...,   0,   0,   0],
       [  0,  -1,   0, ...,   0,   0,   0],
       [  0,   0,  -3, ...,   0,   0,   0],
       ...,
       [  0,   0,   0, ...,  -5,   0,   0],
       [  0,   0,   0, ...,   0,  -5,   0],
       [  0,   0,   0, ...,   0,   0,  -7]], dtype=int8)

In [74]:
# 上三角化
matH_0 = np.triu(matH) + np.tril(matH, k=-1).T

In [75]:
matH_0

array([[-10,   0,   0, ...,   0,   0,   0],
       [  0,  -1,   0, ...,   0,   0,   0],
       [  0,   0,  -3, ...,   0,   0,   0],
       ...,
       [  0,   0,   0, ...,  -5,   0,   0],
       [  0,   0,   0, ...,   0,  -5,   0],
       [  0,   0,   0, ...,   0,   0,  -7]], dtype=int8)

In [76]:
# 下三角の要素が0になっていることの確認
# np.sum(np.tril(matH_0, k=-1))

In [77]:
# numpy配列から辞書を作る方法
# https://www.geeksforgeeks.org/how-to-convert-numpy-array-to-dictionary-in-python/
# d = dict(enumerate(np.arange(4).reshape((2, 2)).ravel(),1))

# 267.2sでMemoryErrorになった
# QUBO_cost2=dict(enumerate(matH.ravel(),1))

In [78]:
QUBO_cost2={}
for i in range(N):
    for j in range(i,N):
        for h in range(H):
            QUBO_cost2[i*H+h,j*H+h] = matH_0[i*H+h,j*H+h]

In [79]:
cost2_sum = sum(QUBO_cost2.values())
beta = H*N/cost2_sum
print('QUBO_cost2の値の合計',cost2_sum)
print('QUBO_cost2の係数β',beta)

QUBO_cost2の値の合計 915164
QUBO_cost2の係数β 0.011178324322197989


matH0(N*H,N*Hの行列)を作らずにmatH1とmatH2_から辞書を作成するパターン（こちらの方が計算量が少ない）

In [80]:
matH1_.ravel()

array([-11,  -2,  -4, ...,  -6,  -6,  -8], dtype=int8)

In [81]:
QUBO_cost2={}
matH1_0 = matH1_.ravel()
matH2_0 = np.triu(matH2_) + np.tril(matH2_, k=-1).T
for i in range(N):
    for j in range(i,N):
        for h in range(H):
            if (i == j):
                QUBO_cost2[i*H+h,j*H+h] = matH1_0[i*H+h]+matH2_0[i,j]
            else:
                QUBO_cost2[i*H+h,j*H+h] = matH2_0[i,j]


In [82]:
cost2_sum = sum(QUBO_cost2.values())
beta = H*N/cost2_sum
print('QUBO_cost2の値の合計',cost2_sum)
print('QUBO_cost2の係数β',beta)

QUBO_cost2の値の合計 915164
QUBO_cost2の係数β 0.011178324322197989


### コスト関数3

In [83]:
# 患者iの現在の病院がhのとき1の行列Hihを定義
# Hih0 = np.zeros(N*H0).reshape(N,H0)
# for i in range(N):
#   for h,hospital_id in enumerate(hospitals.keys()):
#     if unsettled_patients_by_next_bed_type[i].hospital_id == hospital_id:
#       Hih0[i][h] = 1
# Hih0

In [84]:
#病院間の出入りの数がバランスするように調整する項→→病床機能によって急性期→慢性期という一方通行があるため、バランスさせることができない
#cost1 = 0
#for h in range(H):
#  for i in range(N):
#    for j in range(N):
#      cost1 += (Hih[i][h]*Hih[j][h] - 2*Hih[i][h]*Hih[j][h]+x[i,h]*x[j,h])

### トータルのハミルトニアン

In [85]:
# 有効数字で桁落とし
def Round_To_n(x, n):
    return round(x, -int(np.floor(np.sign(x) * np.log10(abs(x)))) + n)


In [86]:

keys_constr1 = QUBO_constr1.keys()
keys_cost1 = QUBO_cost1.keys()
keys_cost2 = QUBO_cost2.keys()
keys = set(list(keys_constr1) + list(keys_cost1) + list(keys_cost2))


print('const1はH*H*N',H*H*N,'要素、対角化して→',len(list(QUBO_constr1.keys())))
print('cost1はH*N',H*N,'要素、対角化して→',len(list(QUBO_cost1.keys())))
print('cost2はH*N*N',H*N*N,'要素、対角化して→',len(list(QUBO_cost2.keys())))
print('各QUBOの要素',len(list(QUBO_constr1.keys()))+len(list(QUBO_cost1.keys()))+len(list(QUBO_cost2.keys())))
print('重複を除く要素',len(keys))
print('重複分はそれぞれの対角成分', H*N,'×2')

const1はH*H*N 634260 要素、対角化して→ 322245
cost1はH*N 10230 要素、対角化して→ 10230
cost2はH*N*N 1687950 要素、対角化して→ 849090
各QUBOの要素 1181565
重複を除く要素 1161105
重複分はそれぞれの対角成分 10230 ×2


In [87]:
alpha = Round_To_n(alpha,3)
beta = Round_To_n(beta,3)
lam1 = 0.5
print('alpha',alpha)
print('beta',beta)
print('lam1',lam1)

alpha 0.02742
beta 0.01118
lam1 0.5


In [92]:
# 上三角行列に変換→samplerへの埋め込み速度向上
'''
QUBO_total = {}
for key in keys:
  key2 = (key[1],key[0]) if key[0]>key[1] else key
  if key2 not in QUBO_total.keys():
    QUBO_total[key2] = 0
  if key2 in keys_cost1:
    QUBO_total[key2] += alpha*QUBO_cost1[key]
  if key2 in keys_cost2:
    QUBO_total[key2] += beta*QUBO_cost2[key]
  if key2 in keys_constr1:
    QUBO_total[key2] += lam1*QUBO_constr1[key]
'''
# 各QUBOを対角化済なので単純に足し算する
QUBO_total = {}
for key in keys:
  if (key in keys_cost1)&(key in keys_cost2)&(key in keys_constr1):#すべてのQUBOで共通の対角成分
    QUBO_total[key] = alpha*QUBO_cost1[key] + beta*QUBO_cost2[key] + lam1*QUBO_constr1[key]
  elif key in keys_cost2:
    QUBO_total[key] = beta*QUBO_cost2[key]
  elif key in keys_constr1:
    QUBO_total[key] = lam1*QUBO_constr1[key]

print(len(keys))
print(len(QUBO_total.keys()))


1161105
1161105


## nealで最適化計算

In [89]:
# neal
import neal
sampler = neal.SimulatedAnnealingSampler()
sampleset_neal = sampler.sample_qubo(QUBO_total, num_reads=50)
print(sampleset_neal)


    0  1  2  3  4  5  6  7  8  9 10 11 12 13 14 15 ... 10229     energy num_oc.
30  0  0  0  0  0  0  0  0  0  0  0  0  0  1  0  0 ...     0 -66.436803       1
16  0  0  0  0  0  0  0  0  0  0  0  1  0  0  0  0 ...     0 -65.766003       1
44  0  0  0  1  0  0  0  0  0  0  0  0  0  0  0  0 ...     0 -65.598303       1
27  0  0  0  0  0  0  0  0  0  0  1  0  0  0  0  0 ...     0 -65.587123       1
33  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0 ...     0 -65.508863       1
26  0  0  0  0  1  0  0  0  0  0  0  0  0  0  0  0 ...     0 -65.441783       1
25  0  0  0  0  1  0  0  0  0  0  0  0  0  0  0  0 ...     0 -65.363523       1
18  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0 ...     0 -65.274083       1
24  0  0  0  0  1  0  0  0  0  0  0  0  0  0  0  0 ...     0 -65.240543       1
31  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0 ...     0 -65.184643       1
11  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0 ...     0 -65.184643       1
5   0  0  1  0  0  0  0  0  0  0  0  0  

In [90]:
#best_solution_neal= list(sampleset_neal.first.sample.values())
best_solution_neal= sampleset_neal.first.sample
print('最適解のエネルギー値', sampleset_neal.first.energy,'constr1の定数項足すと',sampleset_neal.first.energy + lam1*N)#定数項λNを加えたエネルギー（pyQuboと比較用）
print('最適解の発生回数', sampleset_neal.first.num_occurrences,'/',sum(sampleset_neal.data_vectors['num_occurrences']),'回')

# print('最適解',best_solution_neal)

最適解のエネルギー値 -66.43680291448254 constr1の定数項足すと 16.06319708551746
最適解の発生回数 1 / 50 回


In [91]:
'''
# 最適解をマトリックス（行が患者、列が転院先）に変換
best_solution_neal_matrix = np.array(best_solution_neal).reshape(N,H)
for i in range(N):
  print(sum(best_solution_neal_matrix[i][:]))
'''

'\n# 最適解をマトリックス（行が患者、列が転院先）に変換\nbest_solution_neal_matrix = np.array(best_solution_neal).reshape(N,H)\nfor i in range(N):\n  print(sum(best_solution_neal_matrix[i][:]))\n'

In [92]:
# 制約条件に反するエラーがないかどうかの確認。あるとFalseが出る。
constr1_neal = 0
for key in keys_constr1:
  constr1_neal += best_solution_neal[key[0]]*lam1*QUBO_constr1[key]*best_solution_neal[key[1]]
constr1_neal += lam1*N
if constr1_neal == 0:
  print('QUBO_constr1の制約条件OK')
else:
  print('QUBO_constr1の制約条件NG',constr1_neal)

QUBO_constr1の制約条件OK


In [93]:
# 最適解をマトリックス（行が患者、列が転院先）に変換して制約条件を確認
best_solution_neal_matrix = np.array([i[1] for i in sorted(best_solution_neal.items())]).reshape(N,H)

# 行ごとに合計して1（転院先が1つ）になっていることを確認
np.sum(best_solution_neal_matrix,axis=1)

array([1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
       1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
       1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
       1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
       1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
       1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
       1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
       1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1])

In [94]:
# 最適解のエネルギーを数式から確認。sampleset_neal.first.energyと一致するはず。
E = 0
for key in QUBO_total.keys():
  E += best_solution_neal[key[0]]*QUBO_total[key]*best_solution_neal[key[1]]
print(E)

-66.4367999999995


### 答えの表示

In [95]:
# 転院先候補を消去
for i in range(N):
  patient = unsettled_patients_by_next_bed_type[i]
  patient.next_possible_hospital_ids = []

In [96]:
print('転院先の病床:',bed_type,'について',target_date.strftime('%Y/%m/%d'),'までの転院先を最適化しました')
print(N)
for i in range(N):
  patient = unsettled_patients_by_next_bed_type[i]
  lst = patient.next_possible_hospital_ids
  xsum = 0
  for h in range(H):
    if best_solution_neal[i*H+h] == 1:
      #print(patient.expected_discharge_date.strftime('%Y/%m/%d'),'に退院予定の',hospitals[patient.hospital_id].name,'の患者',patient.patient_id,'さんの転院先候補は',hospitals[hospitals_by_empty_bed_type[h]].name,'です')
      # print(patient.expected_discharge_date.strftime('%Y/%m/%d'),'に退院予定の',hospitals[patient.hospital_id].id,'の患者',patient.patient_id,'さんの転院先候補は',hospitals[hospitals_by_empty_bed_type[h]].id,'です')
      print(patient.expected_discharge_date.strftime('%Y/%m/%d'),'に退院予定の',hospitals[patient.hospital_id].id,'の患者',patient.patient_id,'さんの転院先候補は',hospitals[hospitals_by_empty_bed_type[h]].id,'です。',hospitals[patient.hospital_id].second_area,'→',hospitals[hospitals_by_empty_bed_type[h]].second_area)

      # print(hospitals_by_empty_bed_type[h])
      lst.append(hospitals_by_empty_bed_type[h])
      # print(lst)
      lst = list(set(lst))
      # print(lst)
      xsum += 1
  if xsum == 0:
    print(patient.expected_discharge_date.strftime('%Y/%m/%d'),'に退院予定の',hospitals[patient.hospital_id].name,'の患者',patient.patient_id,'さんの転院先候補はみつかりませんでした')

  # 転院先候補を書き込み
  patient.next_possible_hospital_ids = lst


転院先の病床: 慢性期 について 2021/08/10 までの転院先を最適化しました
165
2021/08/10 に退院予定の H00001 の患者 慢性期0008 さんの転院先候補は H00039 です。 仙台医療圏 → 仙台医療圏
2021/08/10 に退院予定の H00001 の患者 慢性期0010 さんの転院先候補は H00015 です。 仙台医療圏 → 仙台医療圏
2021/08/07 に退院予定の H00001 の患者 慢性期0027 さんの転院先候補は H00036 です。 仙台医療圏 → 仙台医療圏
2021/08/10 に退院予定の H00002 の患者 慢性期0010 さんの転院先候補は H00015 です。 仙台医療圏 → 仙台医療圏
2021/08/10 に退院予定の H00002 の患者 慢性期0011 さんの転院先候補は H00039 です。 仙台医療圏 → 仙台医療圏
2021/08/08 に退院予定の H00002 の患者 慢性期0027 さんの転院先候補は H00039 です。 仙台医療圏 → 仙台医療圏
2021/08/07 に退院予定の H00002 の患者 慢性期0033 さんの転院先候補は H00018 です。 仙台医療圏 → 仙台医療圏
2021/08/09 に退院予定の H00007 の患者 慢性期0006 さんの転院先候補は H00039 です。 仙台医療圏 → 仙台医療圏
2021/08/09 に退院予定の H00007 の患者 慢性期0046 さんの転院先候補は H00027 です。 仙台医療圏 → 仙台医療圏
2021/08/09 に退院予定の H00007 の患者 慢性期0069 さんの転院先候補は H00015 です。 仙台医療圏 → 仙台医療圏
2021/08/07 に退院予定の H00007 の患者 慢性期0082 さんの転院先候補は H00027 です。 仙台医療圏 → 仙台医療圏
2021/08/10 に退院予定の H00013 の患者 慢性期0033 さんの転院先候補は H00015 です。 仙台医療圏 → 仙台医療圏
2021/08/07 に退院予定の H00015 の患者 慢性期0033 さんの転院先候補は H00039 です。 仙台医療圏 → 仙台医療圏
2021/08/07 に退院予定の

In [97]:
# empty_beds_by_typesをコピーして最適化後の結果を確認。3重の辞書形式なのでdeepcopyを使う。
import copy
empty_beds_by_types_optimized_neal = copy.deepcopy(empty_beds_by_types)


# 制約条件が守られているときだけ実行
if constr1_neal == 0:
  print('受け入れ病院の、転院日より先の日付の空き病床数をマイナス-1')
  for patient in unsettled_patients_by_next_bed_type:
    print(patient.patient_id, patient.next_possible_hospital_ids)

    # 候補が1個以上あるときは1つ目の要素を選択。※候補がない場合は制約条件違反
    if len(patient.next_possible_hospital_ids) > 0:
      h = patient.next_possible_hospital_ids[0]

      # patientが転院先の第一候補に転院する際の空き病床数を確認
      for d, empty_bed_num in empty_beds_by_types_optimized_neal[bed_type][h].items():
        #print(date,empty_bed_num)
        # 転院者受け入れ日以降の日付の空き病床数をマイナス-1
        if d >= patient.expected_discharge_date:
          empty_bed_num = empty_bed_num -1

        # 結果をtempt反映
        empty_beds_by_types_optimized_neal[bed_type][h][d] = empty_bed_num

        # 結果を病院のクラスに反映
        # hospitals[hospital].empty_beds[date][bed_type] = empty_bed_num

else:
  print('制約条件を守る解はみつかりませんでした')

受け入れ病院の、転院日より先の日付の空き病床数をマイナス-1
慢性期0008 ['H00039']
慢性期0010 ['H00015']
慢性期0027 ['H00036']
慢性期0010 ['H00015']
慢性期0011 ['H00039']
慢性期0027 ['H00039']
慢性期0033 ['H00018']
慢性期0006 ['H00039']
慢性期0046 ['H00027']
慢性期0069 ['H00015']
慢性期0082 ['H00027']
慢性期0033 ['H00015']
慢性期0033 ['H00039']
慢性期0035 ['H00027']
慢性期0067 ['H00039']
慢性期0072 ['H00057']
慢性期0082 ['H00039']
慢性期0099 ['H00046']
慢性期0107 ['H00055']
慢性期0128 ['H00039']
慢性期0140 ['H00046']
慢性期0172 ['H00036']
慢性期0180 ['H00046']
慢性期0008 ['H00039']
慢性期0001 ['H00049']
慢性期0015 ['H00058']
慢性期0028 ['H00046']
慢性期0003 ['H00039']
慢性期0013 ['H00051']
慢性期0003 ['H00057']
慢性期0012 ['H00002']
慢性期0019 ['H00057']
慢性期0026 ['H00039']
慢性期0073 ['H00039']
慢性期0122 ['H00057']
慢性期0141 ['H00047']
慢性期0151 ['H00057']
慢性期0175 ['H00057']
慢性期0201 ['H00015']
慢性期0258 ['H00057']
慢性期0014 ['H00027']
慢性期0018 ['H00013']
慢性期0035 ['H00050']
慢性期0005 ['H00015']
慢性期0043 ['H00032']
慢性期0045 ['H00057']
慢性期0051 ['H00057']
慢性期0053 ['H00057']
慢性期0093 ['H00015']
慢性期0109 ['H00063']
慢性期0003 ['H00058']


In [98]:
print(hospitals_by_empty_bed_type)
print(hospital_ids_of_patients_by_next_bed_type)
target_hospitals = hospital_ids_of_patients_by_next_bed_type.copy()
target_hospitals.update(hospitals_by_empty_bed_type)
target_hospitals = sorted(target_hospitals, key=str)
print(target_hospitals)

['H00001', 'H00002', 'H00007', 'H00013', 'H00015', 'H00018', 'H00020', 'H00023', 'H00027', 'H00031', 'H00032', 'H00035', 'H00036', 'H00039', 'H00043', 'H00046', 'H00047', 'H00049', 'H00050', 'H00051', 'H00052', 'H00055', 'H00057', 'H00058', 'H00059', 'H00060', 'H00063', 'H00064', 'H00066', 'H00067', 'H00069', 'H00073', 'H00083', 'H00084', 'H00085', 'H00086', 'H00087', 'H00088', 'H00090', 'H00093', 'H00094', 'H00095', 'H00098', 'H00099', 'H00100', 'H00101', 'H00103', 'H00109', 'H00118', 'H00121', 'H00123', 'H00124', 'H00125', 'H00127', 'H00132', 'H00133', 'H00134', 'H00136', 'H00138', 'H00140', 'H00145', 'H00150']
{'H00051', 'H00066', 'H00136', 'H00018', 'H00020', 'H00067', 'H00027', 'H00073', 'H00039', 'H00090', 'H00050', 'H00063', 'H00001', 'H00058', 'H00086', 'H00124', 'H00069', 'H00060', 'H00043', 'H00007', 'H00047', 'H00098', 'H00103', 'H00093', 'H00059', 'H00099', 'H00057', 'H00064', 'H00095', 'H00121', 'H00123', 'H00138', 'H00145', 'H00088', 'H00133', 'H00085', 'H00150', 'H00101'

In [99]:
print('最適化後の',bed_type,'の病床変化')
violation = (0,[])
for h in target_hospitals:
  if h in hospital_ids_of_patients_by_next_bed_type:
    print('転院対象患者あり')
  if h in hospitals_by_empty_bed_type:
    print('受け入れ病床あり')
  for d,b in empty_beds_by_types[bed_type][h].items():
    if d >= first_date:
      b1 = b if d == first_date else b1
      b2 = empty_beds_by_types_optimized_neal[bed_type][h][d]
      violation = (violation[0] + 1, violation[1]+[h]) if b2 < 0 else violation
      violation_text = '病床マイナス' if b2 < 0 else ''
      print(h,d,b,"->",b2,'(',b1-b2,')',violation_text)
      b1 = b2
  print('------------------------------------')
print('病床マイナス',violation)

最適化後の 慢性期 の病床変化
転院対象患者あり
受け入れ病床あり
H00001 2021-08-06 00:00:00 11 -> 11 ( 0 ) 
H00001 2021-08-07 00:00:00 11 -> 11 ( 0 ) 
H00001 2021-08-08 00:00:00 11 -> 10 ( 1 ) 
H00001 2021-08-09 00:00:00 11 -> 10 ( 0 ) 
H00001 2021-08-10 00:00:00 11 -> 10 ( 0 ) 
------------------------------------
転院対象患者あり
受け入れ病床あり
H00002 2021-08-06 00:00:00 2 -> 2 ( 0 ) 
H00002 2021-08-07 00:00:00 2 -> 2 ( 0 ) 
H00002 2021-08-08 00:00:00 2 -> 2 ( 0 ) 
H00002 2021-08-09 00:00:00 2 -> 1 ( 1 ) 
H00002 2021-08-10 00:00:00 2 -> 1 ( 0 ) 
------------------------------------
転院対象患者あり
受け入れ病床あり
H00007 2021-08-06 00:00:00 4 -> 4 ( 0 ) 
H00007 2021-08-07 00:00:00 4 -> 3 ( 1 ) 
H00007 2021-08-08 00:00:00 4 -> 3 ( 0 ) 
H00007 2021-08-09 00:00:00 4 -> 3 ( 0 ) 
H00007 2021-08-10 00:00:00 4 -> 3 ( 0 ) 
------------------------------------
転院対象患者あり
受け入れ病床あり
H00013 2021-08-06 00:00:00 9 -> 9 ( 0 ) 
H00013 2021-08-07 00:00:00 9 -> 9 ( 0 ) 
H00013 2021-08-08 00:00:00 9 -> 8 ( 1 ) 
H00013 2021-08-09 00:00:00 9 -> 7 ( 1 ) 
H00013 2021-

In [105]:
# 最適化後のベッドの空き状況を出力
for h,dates in empty_beds_by_types_optimized_neal[bed_type].items():
  for date,bed in dates.items():
    db_empty_beds.loc[(db_empty_beds['hospital_id']==h)&(db_empty_beds['date']==date),bed_type] = bed

db_empty_beds.to_csv('./output/db_empty_beds_neal.csv')



In [108]:
for patient in unsettled_patients_by_next_bed_type:
  db_patients.loc[(db_patients['hospital_id']==patient.current_hospital_id) & (db_patients['patient_id'] == patient.patient_id),'next_hospital_id'] = patient.next_possible_hospital_ids[0]
db_patients.to_csv('./output/DB_Patients_neal.csv')

## Openjijで最適化計算

In [93]:
from openjij import SQASampler
sampler = SQASampler()
sampleset_openjij = sampler.sample_qubo(QUBO_total,num_reads=10)
print(sampleset_openjij)

   0  1  2  3  4  5  6  7  8  9 10 11 12 13 14 15 ... 10229    energy num_oc.
0  0  0  0  0  0  0  0  0  0  0  0  0  0  1  0  0 ...     0 -33.65978       1
9  0  0  1  0  0  0  0  0  0  0  0  0  0  0  0  0 ...     0   -31.885       1
1  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0 ...     0 -30.60724       1
5  0  0  0  1  0  0  0  0  0  0  0  0  0  0  0  0 ...     0 -29.67304       1
7  0  0  0  0  0  0  0  1  0  0  0  0  0  0  0  0 ...     0 -29.05518       1
6  0  0  0  0  1  0  0  0  0  0  0  0  0  0  0  0 ...     0 -24.64476       1
3  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0 ...     0 -24.05976       1
4  0  0  0  0  0  0  0  1  0  0  0  0  0  0  0  0 ...     0 -23.30536       1
2  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0 ...     0 -22.53308       1
8  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0 ...     0 -21.68374       1
['BINARY', 10 rows, 10 samples, 10230 variables]


In [94]:
print(sampleset_openjij)

   0  1  2  3  4  5  6  7  8  9 10 11 12 13 14 15 ... 10229    energy num_oc.
0  0  0  0  0  0  0  0  0  0  0  0  0  0  1  0  0 ...     0 -33.65978       1
9  0  0  1  0  0  0  0  0  0  0  0  0  0  0  0  0 ...     0   -31.885       1
1  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0 ...     0 -30.60724       1
5  0  0  0  1  0  0  0  0  0  0  0  0  0  0  0  0 ...     0 -29.67304       1
7  0  0  0  0  0  0  0  1  0  0  0  0  0  0  0  0 ...     0 -29.05518       1
6  0  0  0  0  1  0  0  0  0  0  0  0  0  0  0  0 ...     0 -24.64476       1
3  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0 ...     0 -24.05976       1
4  0  0  0  0  0  0  0  1  0  0  0  0  0  0  0  0 ...     0 -23.30536       1
2  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0 ...     0 -22.53308       1
8  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0 ...     0 -21.68374       1
['BINARY', 10 rows, 10 samples, 10230 variables]


In [95]:
sampleset_openjij.record[0][0]

array([0, 0, 0, ..., 0, 0, 0], dtype=int8)

In [96]:
best_solution_openjij= sampleset_openjij.first.sample
print('最適解のエネルギー値', sampleset_openjij.first.energy,'constr1の定数項足すと',sampleset_openjij.first.energy + lam1*N) #定数項λNを加えたエネルギー（pyQuboと比較用）
print('最適解の発生回数', sampleset_openjij.first.num_occurrences,'/',sum(sampleset_openjij.data_vectors['num_occurrences']),'回')
#print('最適解',best_solution_openjij)

最適解のエネルギー値 -33.65977999720607 constr1の定数項足すと 48.84022000279393
最適解の発生回数 1 / 10 回


In [97]:
# 制約条件に反するエラーがないかどうかの確認。あるとFalseが出る。
constr1_openjij = 0
for key in keys_constr1:
  constr1_openjij += best_solution_openjij[key[0]]*lam1*QUBO_constr1[key]*best_solution_openjij[key[1]]
constr1_openjij += lam1*N
if constr1_openjij == 0:
  print('QUBO_constr1の制約条件OK')
else:
  print('QUBO_constr1の制約条件NG',constr1_openjij)

QUBO_constr1の制約条件NG 17.5


In [98]:
# 最適解のエネルギーを数式から確認。sampleset_openjij.first.energyと一致するはず。
E = 0
for key in QUBO_total.keys():
  E += best_solution_openjij[key[0]]*QUBO_total[key]*best_solution_openjij[key[1]]
print(E)

-33.65978000000008


In [99]:
# 最適解をマトリックス（行が患者、列が転院先）に変換して制約条件を確認
best_solution_openjij_matrix = np.array([i[1] for i in sorted(best_solution_openjij.items())]).reshape(N,H)

# 行ごとに合計して1（転院先が1つ）になっていることを確認
np.sum(best_solution_openjij_matrix, axis=1)

array([1, 1, 1, 1, 1, 1, 1, 1, 1, 2, 1, 1, 1, 1, 1, 2, 1, 0, 2, 1, 2, 2,
       1, 1, 1, 1, 2, 1, 1, 1, 2, 1, 1, 1, 1, 1, 1, 2, 1, 1, 1, 1, 1, 1,
       2, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 2, 2, 1, 1, 1, 2, 1, 1, 1, 1, 1,
       1, 1, 1, 1, 2, 2, 1, 1, 1, 1, 1, 2, 2, 1, 1, 2, 1, 1, 1, 2, 1, 1,
       1, 1, 1, 1, 1, 2, 1, 1, 1, 1, 1, 1, 1, 2, 1, 1, 2, 2, 1, 1, 1, 1,
       1, 1, 1, 1, 1, 2, 1, 1, 1, 1, 2, 1, 2, 2, 2, 2, 1, 1, 1, 1, 1, 2,
       1, 2, 1, 1, 1, 2, 1, 1, 1, 1, 1, 1, 1, 1, 1, 2, 1, 1, 1, 1, 1, 2,
       1, 1, 1, 1, 1, 1, 1, 1, 1, 2, 1])

### Openjijの最適化結果を表示。患者のインスタンスの転院先候補を更新

In [100]:
# 転院先候補を消去
for i in range(N):
  patient = unsettled_patients_by_next_bed_type[i]
  patient.next_possible_hospital_ids = []

In [101]:
print('転院先の病床:',bed_type,'について',target_date.strftime('%Y/%m/%d'),'までの転院先を最適化しました')
for i in range(N):
  patient = unsettled_patients_by_next_bed_type[i]
  lst = patient.next_possible_hospital_ids
  # print(lst)
  xsum = 0
  for h in range(H):
    if best_solution_openjij[i*H+h] == 1:
      # print(patient.expected_discharge_date.strftime('%Y/%m/%d'),'に退院予定の',hospitals[patient.hospital_id].name,'の患者',patient.patient_id,'さんの転院先候補は',hospitals[hospitals_by_empty_bed_type[h]].name,'です')
      print(patient.expected_discharge_date.strftime('%Y/%m/%d'),'に退院予定の',hospitals[patient.hospital_id].id,'の患者',patient.patient_id,'さんの転院先候補は',hospitals[hospitals_by_empty_bed_type[h]].id,'です。',hospitals[patient.hospital_id].second_area,'→',hospitals[hospitals_by_empty_bed_type[h]].second_area)

      # print(hospitals_by_empty_bed_type[h])
      lst.append(hospitals_by_empty_bed_type[h])
      # print(lst)
      lst = list(set(lst))
      # print(lst)
      xsum += 1

  if xsum == 0:
    print(patient.expected_discharge_date.strftime('%Y/%m/%d'),'に退院予定の',hospitals[patient.hospital_id].id,'の患者',patient.patient_id,'さんの転院先候補はみつかりませんでした')

  # 転院先候補を書き込み
  patient.next_possible_hospital_ids = lst


転院先の病床: 慢性期 について 2021/08/10 までの転院先を最適化しました
2021/08/10 に退院予定の H00001 の患者 慢性期0008 さんの転院先候補は H00039 です。 仙台医療圏 → 仙台医療圏
2021/08/10 に退院予定の H00001 の患者 慢性期0010 さんの転院先候補は H00020 です。 仙台医療圏 → 仙台医療圏
2021/08/07 に退院予定の H00001 の患者 慢性期0027 さんの転院先候補は H00020 です。 仙台医療圏 → 仙台医療圏
2021/08/10 に退院予定の H00002 の患者 慢性期0010 さんの転院先候補は H00031 です。 仙台医療圏 → 仙台医療圏
2021/08/10 に退院予定の H00002 の患者 慢性期0011 さんの転院先候補は H00013 です。 仙台医療圏 → 仙台医療圏
2021/08/08 に退院予定の H00002 の患者 慢性期0027 さんの転院先候補は H00039 です。 仙台医療圏 → 仙台医療圏
2021/08/07 に退院予定の H00002 の患者 慢性期0033 さんの転院先候補は H00020 です。 仙台医療圏 → 仙台医療圏
2021/08/09 に退院予定の H00007 の患者 慢性期0006 さんの転院先候補は H00052 です。 仙台医療圏 → 仙台医療圏
2021/08/09 に退院予定の H00007 の患者 慢性期0046 さんの転院先候補は H00001 です。 仙台医療圏 → 仙台医療圏
2021/08/09 に退院予定の H00007 の患者 慢性期0069 さんの転院先候補は H00015 です。 仙台医療圏 → 仙台医療圏
2021/08/09 に退院予定の H00007 の患者 慢性期0069 さんの転院先候補は H00023 です。 仙台医療圏 → 仙台医療圏
2021/08/07 に退院予定の H00007 の患者 慢性期0082 さんの転院先候補は H00035 です。 仙台医療圏 → 仙台医療圏
2021/08/10 に退院予定の H00013 の患者 慢性期0033 さんの転院先候補は H00046 です。 仙台医療圏 → 仙台医療圏
2021/08/07 に退院予定の H00

### Openjijの最適化結果をもとに空き病床数をアップデート

In [102]:
# empty_beds_by_typesをコピーして最適化後の結果を確認。3重の辞書形式なのでdeepcopyを使う。
import copy
empty_beds_by_types_optimized = copy.deepcopy(empty_beds_by_types)


# 制約条件が守られているときだけ実行
if constr1_openjij == 0:
  print('受け入れ病院の、転院日より先の日付の空き病床数をマイナス-1')
  for patient in unsettled_patients_by_next_bed_type:
    if (patient.current_hospital_id == patient.next_possible_hospital_ids[0]) and (patient.current_bed_type == bed_type):
      print(patient.patient_id, patient.current_hospital_id, '→', patient.next_possible_hospital_ids, '←×同じ病院に転院')
    elif (patient.current_hospital_id == patient.next_possible_hospital_ids[0]) and (patient.current_bed_type != bed_type):
      print(patient.patient_id, patient.current_hospital_id, '→', patient.next_possible_hospital_ids, '←○病床タイプだけ変わるパターンで、同じ病院に転院')
    elif (patient.current_hospital_id != patient.next_possible_hospital_ids[0]) and (patient.current_bed_type != bed_type):
      print(patient.patient_id, patient.current_hospital_id, '→', patient.next_possible_hospital_ids, '←△病床タイプだけ変わるパターンだが、違う病院に転院')
    else:
      print(patient.patient_id, patient.current_hospital_id, patient.current_bed_type,'→', patient.next_possible_hospital_ids)

    # 候補が1個以上あるときは1つ目の要素を選択。※候補がない場合は制約条件違反
    if len(patient.next_possible_hospital_ids) > 0:
      h = patient.next_possible_hospital_ids[0]

      # patientが転院先の第一候補に転院する際の空き病床数を確認
      for d, empty_bed_num in empty_beds_by_types_optimized[bed_type][h].items():
        #print(date,empty_bed_num)
        # 転院者受け入れ日以降の日付の空き病床数をマイナス-1
        if d >= patient.expected_discharge_date:
          empty_bed_num = empty_bed_num -1

        # 結果をtempt反映
        empty_beds_by_types_optimized[bed_type][h][d] = empty_bed_num

        # 結果を病院のクラスに反映
        # hospitals[hospital].empty_beds[date][bed_type] = empty_bed_num

else:
  print('制約条件を守る解はみつかりませんでした')




制約条件を守る解はみつかりませんでした


### 答えを表示

In [103]:
#対象ベッドタイプ、対象期間の空きベッド状況
#for h in hospitals.values():
#  print(h.get_empty_beds_by_type(date, bed_type))

In [105]:
print(hospitals_by_empty_bed_type)
print(hospital_ids_of_patients_by_next_bed_type)
target_hospitals = hospital_ids_of_patients_by_next_bed_type.copy()
target_hospitals.update(hospitals_by_empty_bed_type)
target_hospitals = sorted(target_hospitals, key=str)
print(target_hospitals)

['H00001', 'H00002', 'H00007', 'H00013', 'H00015', 'H00018', 'H00020', 'H00023', 'H00027', 'H00031', 'H00032', 'H00035', 'H00036', 'H00039', 'H00043', 'H00046', 'H00047', 'H00049', 'H00050', 'H00051', 'H00052', 'H00055', 'H00057', 'H00058', 'H00059', 'H00060', 'H00063', 'H00064', 'H00066', 'H00067', 'H00069', 'H00073', 'H00083', 'H00084', 'H00085', 'H00086', 'H00087', 'H00088', 'H00090', 'H00093', 'H00094', 'H00095', 'H00098', 'H00099', 'H00100', 'H00101', 'H00103', 'H00109', 'H00118', 'H00121', 'H00123', 'H00124', 'H00125', 'H00127', 'H00132', 'H00133', 'H00134', 'H00136', 'H00138', 'H00140', 'H00145', 'H00150']
{'H00095', 'H00002', 'H00101', 'H00083', 'H00086', 'H00047', 'H00132', 'H00035', 'H00069', 'H00067', 'H00013', 'H00036', 'H00057', 'H00084', 'H00100', 'H00073', 'H00099', 'H00052', 'H00020', 'H00123', 'H00145', 'H00087', 'H00127', 'H00058', 'H00027', 'H00043', 'H00063', 'H00136', 'H00015', 'H00051', 'H00133', 'H00039', 'H00046', 'H00032', 'H00103', 'H00060', 'H00098', 'H00064'

In [106]:
print('転院先の病床:',bed_type,'について',target_date.strftime('%Y/%m/%d'),'までに転院する',N,'人の転院先を',H,'種類の病院から最適化しました')
print('最適化後の',bed_type,'の病床変化')
violation = (0,[])
for h in target_hospitals:
  if h in hospital_ids_of_patients_by_next_bed_type:
    print('転院対象患者あり')
  if h in hospitals_by_empty_bed_type:
    print('受け入れ病床あり')
  for d,b in empty_beds_by_types[bed_type][h].items():
    if d >= first_date:
      b1 = b if d == first_date else b1
      b2 = empty_beds_by_types_optimized[bed_type][h][d]
      violation = (violation[0] + 1, violation[1]+[h]) if b2 < 0 else violation
      violation_text = '病床マイナス' if b2 < 0 else ''
      print(h,d,b,"->",b2,'(',b1-b2,')',violation_text)
      b1 = b2
  print('------------------------------------')
print('病床マイナス',violation)


転院先の病床: 慢性期 について 2021/08/10 までに転院する 165 人の転院先を 62 種類の病院から最適化しました
最適化後の 慢性期 の病床変化
転院対象患者あり
受け入れ病床あり
H00001 2021-08-06 00:00:00 11 -> 11 ( 0 ) 
H00001 2021-08-07 00:00:00 11 -> 11 ( 0 ) 
H00001 2021-08-08 00:00:00 11 -> 11 ( 0 ) 
H00001 2021-08-09 00:00:00 11 -> 11 ( 0 ) 
H00001 2021-08-10 00:00:00 11 -> 11 ( 0 ) 
------------------------------------
転院対象患者あり
受け入れ病床あり
H00002 2021-08-06 00:00:00 2 -> 2 ( 0 ) 
H00002 2021-08-07 00:00:00 2 -> 2 ( 0 ) 
H00002 2021-08-08 00:00:00 2 -> 2 ( 0 ) 
H00002 2021-08-09 00:00:00 2 -> 2 ( 0 ) 
H00002 2021-08-10 00:00:00 2 -> 2 ( 0 ) 
------------------------------------
転院対象患者あり
受け入れ病床あり
H00007 2021-08-06 00:00:00 4 -> 4 ( 0 ) 
H00007 2021-08-07 00:00:00 4 -> 4 ( 0 ) 
H00007 2021-08-08 00:00:00 4 -> 4 ( 0 ) 
H00007 2021-08-09 00:00:00 4 -> 4 ( 0 ) 
H00007 2021-08-10 00:00:00 4 -> 4 ( 0 ) 
------------------------------------
転院対象患者あり
受け入れ病床あり
H00013 2021-08-06 00:00:00 9 -> 9 ( 0 ) 
H00013 2021-08-07 00:00:00 9 -> 9 ( 0 ) 
H00013 2021-08-08 00:00:00 9 

## D-Waveで最適化計算

In [89]:
# 下記はエラーになる
'''
from dwave.system import DWaveSampler, EmbeddingComposite

dw_sampler = DWaveSampler(solver='Advantage_system1.1', token=token, endpoint=endpoint)
sampler = EmbeddingComposite(dw_sampler)
sampleset_dw = sampler.sample_qubo(qubo,num_reads=10)
'''
'''エラー内容
minorminer/_minorminer.pyx in minorminer._minorminer.find_embedding()
RuntimeError: Failed during initialization.  This typically occurs when the source graph is unreasonably large or when the embedding problem is over-constrained (via max_fill, initial_chains, fixed_chains, and/or restrict_chains).
'''

'エラー内容\nminorminer/_minorminer.pyx in minorminer._minorminer.find_embedding()\nRuntimeError: Failed during initialization.  This typically occurs when the source graph is unreasonably large or when the embedding problem is over-constrained (via max_fill, initial_chains, fixed_chains, and/or restrict_chains).\n'

### トータルのハミルトニアン調整

In [95]:
alpha = Round_To_n(alpha,3)
beta = Round_To_n(beta,3)
lam1 = 0.5
print('alpha',alpha)
print('beta',beta)
print('lam1',lam1)

alpha 0.02742
beta 0.01118
lam1 0.5


In [91]:
# 上三角行列に変換→samplerへの埋め込み速度向上
'''
QUBO_total = {}
for key in keys:
  key2 = (key[1],key[0]) if key[0]>key[1] else key
  if key2 not in QUBO_total.keys():
    QUBO_total[key2] = 0
  if key2 in keys_cost1:
    QUBO_total[key2] += alpha*QUBO_cost1[key]
  if key2 in keys_cost2:
    QUBO_total[key2] += beta*QUBO_cost2[key]
  if key2 in keys_constr1:
    QUBO_total[key2] += lam1*QUBO_constr1[key]
'''
'''
# 上の方で作成済み
# 各QUBOを対角化済なので単純に足し算する
QUBO_total = {}
for key in keys:
  if (key in keys_cost1)&(key in keys_cost2)&(key in keys_constr1):#すべてのQUBOで共通の対角成分
    QUBO_total[key] = alpha*QUBO_cost1[key] + beta*QUBO_cost2[key] + lam1*QUBO_constr1[key]
  elif key in keys_cost2:
    QUBO_total[key] = beta*QUBO_cost2[key]
  elif key in keys_constr1:
    QUBO_total[key] = lam1*QUBO_constr1[key]

print(len(keys))
print(len(QUBO_total.keys()))
print(sum(QUBO_total.values()))
'''

'\n# 上の方で作成済み\n# 各QUBOを対角化済なので単純に足し算する\nQUBO_total = {}\nfor key in keys:\n  if (key in keys_cost1)&(key in keys_cost2)&(key in keys_constr1):#すべてのQUBOで共通の対角成分\n    QUBO_total[key] = alpha*QUBO_cost1[key] + beta*QUBO_cost2[key] + lam1*QUBO_constr1[key]\n  elif key in keys_cost2:\n    QUBO_total[key] = beta*QUBO_cost2[key]\n  elif key in keys_constr1:\n    QUBO_total[key] = lam1*QUBO_constr1[key]\n\nprint(len(keys))\nprint(len(QUBO_total.keys()))\nprint(sum(QUBO_total.values()))\n'


### 要素が多すぎるせいか、Embeddingエラーが出るのでハイブリッドシステムで計算

In [96]:
from dwave.system import LeapHybridSampler

In [ ]:
from config import DWAVE_API_TOKEN
token = DWAVE_API_TOKEN
endpoint = 'https://cloud.dwavesys.com/sapi/'
sampler = LeapHybridSampler(solver='hybrid_binary_quadratic_model_version2', token=token, endpoint=endpoint)

In [ ]:
sampleset_dw = sampler.sample_qubo(QUBO_total)

In [ ]:
print(sampleset_dw)

In [ ]:
#keyでsortされていない
#sampleset_dw.first.sample
#sampleset_dw.record
'''
rec.array([([0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,…, 0], -67.78957866, 1)],
          dtype=[('sample', 'i1', (10230,)), ('energy', '<f8'), ('num_occurrences', '<i8')])'''

In [ ]:
#keyでソートしてリストにする
#temp = [i[1] for i in sorted(sampleset_dw.first.sample.items())]


In [ ]:
best_solution_dw = sampleset_dw.first.sample
print('最適解のエネルギー値', sampleset_dw.first.energy,'constr1の定数項足すと',sampleset_dw.first.energy + lam1*N)
print('最適解の発生回数', sampleset_dw.first.num_occurrences,'/',sum(sampleset_dw.data_vectors['num_occurrences']),'回')
# print('最適解',best_solution_dw)

In [ ]:
# 制約条件に反するエラーがないかどうかの確認。あるとFalseが出る。
constr1_dw = 0
for key in keys_constr1:
  constr1_dw += best_solution_dw[key[0]]*lam1*QUBO_constr1[key]*best_solution_dw[key[1]]
constr1_dw += lam1*N
if constr1_dw == 0:
  print('QUBO_constr1の制約条件OK')
else:
  print('QUBO_constr1の制約条件NG',constr1_dw)

In [ ]:
# 最適解をマトリックス（行が患者、列が転院先）に変換して制約条件を確認
best_solution_dw_matrix = np.array([i[1] for i in sorted(best_solution_dw.items())]).reshape(N,H)
np.sum(best_solution_dw_matrix,axis=1)

In [ ]:
# 最適解のエネルギーを数式から確認。sampleset_dw.first.energyと一致するはず。
E = 0
for key in QUBO_total.keys():
  E += best_solution_dw[key[0]]*QUBO_total[key]*best_solution_dw[key[1]]
print(E)

### D-Waveの最適化結果を表示。患者のインスタンスの転院先候補を更新

In [ ]:
# 転院先候補を消去
for i in range(N):
  patient = unsettled_patients_by_next_bed_type[i]
  patient.next_possible_hospital_ids = []

In [ ]:
print('転院先の病床:',bed_type,'について',target_date.strftime('%Y/%m/%d'),'までの転院先を最適化しました')
for i in range(N):
  patient = unsettled_patients_by_next_bed_type[i]
  lst = patient.next_possible_hospital_ids
  # print(lst)
  xsum = 0
  for h in range(H):
    if best_solution_dw[i*H+h] == 1:
      # print(patient.expected_discharge_date.strftime('%Y/%m/%d'),'に退院予定の',hospitals[patient.hospital_id].name,'の患者',patient.patient_id,'さんの転院先候補は',hospitals[hospitals_by_empty_bed_type[h]].name,'です')
      print(patient.expected_discharge_date.strftime('%Y/%m/%d'),'に退院予定の',hospitals[patient.hospital_id].id,'の患者',patient.patient_id,'さんの転院先候補は',hospitals[hospitals_by_empty_bed_type[h]].id,'です。',hospitals[patient.hospital_id].second_area,'→',hospitals[hospitals_by_empty_bed_type[h]].second_area)

      # print(hospitals_by_empty_bed_type[h])
      lst.append(hospitals_by_empty_bed_type[h])
      # print(lst)
      lst = list(set(lst))
      # print(lst)
      xsum += 1

  if xsum == 0:
    print(patient.expected_discharge_date.strftime('%Y/%m/%d'),'に退院予定の',hospitals[patient.hospital_id].id,'の患者',patient.patient_id,'さんの転院先候補はみつかりませんでした')

  # 転院先候補を書き込み
  patient.next_possible_hospital_ids = lst


### D-Waveの最適化結果をもとに空き病床数をアップデート

In [ ]:
# empty_beds_by_typesをコピーして最適化後の結果を確認。3重の辞書形式なのでdeepcopyを使う。
import copy
empty_beds_by_types_optimized_dw = copy.deepcopy(empty_beds_by_types)


# 制約条件が守られているときだけ実行
if constr1_dw == 0:
  print('受け入れ病院の、転院日より先の日付の空き病床数をマイナス-1')
  for patient in unsettled_patients_by_next_bed_type:
    if (patient.current_hospital_id == patient.next_possible_hospital_ids[0]) and (patient.current_bed_type == bed_type):
      print(patient.patient_id, patient.current_hospital_id, '→', patient.next_possible_hospital_ids, '←×同じ病院に転院')
    elif (patient.current_hospital_id == patient.next_possible_hospital_ids[0]) and (patient.current_bed_type != bed_type):
      print(patient.patient_id, patient.current_hospital_id, '→', patient.next_possible_hospital_ids, '←○病床タイプだけ変わるパターンで、同じ病院に転院')
    elif (patient.current_hospital_id != patient.next_possible_hospital_ids[0]) and (patient.current_bed_type != bed_type):
      print(patient.patient_id, patient.current_hospital_id, '→', patient.next_possible_hospital_ids, '←△病床タイプだけ変わるパターンだが、違う病院に転院')
    else:
      print(patient.patient_id, patient.current_hospital_id, patient.current_bed_type,'→', patient.next_possible_hospital_ids)

    # 候補が1個以上あるときは1つ目の要素を選択。※候補がない場合は制約条件違反
    if len(patient.next_possible_hospital_ids) > 0:
      h = patient.next_possible_hospital_ids[0]

      # patientが転院先の第一候補に転院する際の空き病床数を確認
      for d, empty_bed_num in empty_beds_by_types_optimized_dw[bed_type][h].items():
        #print(date,empty_bed_num)
        # 転院者受け入れ日以降の日付の空き病床数をマイナス-1
        if d >= patient.expected_discharge_date:
          empty_bed_num = empty_bed_num -1

        # 結果をtempt反映
        empty_beds_by_types_optimized_dw[bed_type][h][d] = empty_bed_num

        # 結果を病院のクラスに反映
        # hospitals[hospital].empty_beds[date][bed_type] = empty_bed_num

else:
  print('制約条件を守る解はみつかりませんでした')




### 答えを表示

In [ ]:
print('転院先の病床:',bed_type,'について',target_date.strftime('%Y/%m/%d'),'までに転院する',N,'人の転院先を',H,'種類の病院から最適化しました')
print('最適化後の',bed_type,'の病床変化')
violation = (0,[])
for h in target_hospitals:
  if h in hospital_ids_of_patients_by_next_bed_type:
    print('転院対象患者あり')
  if h in hospitals_by_empty_bed_type:
    print('受け入れ病床あり')
  for d,b in empty_beds_by_types[bed_type][h].items():
    if d >= first_date:
      b1 = b if d == first_date else b1
      b2 = empty_beds_by_types_optimized_dw[bed_type][h][d]
      violation = (violation[0] + 1, violation[1]+[h]) if b2 < 0 else violation
      violation_text = '病床マイナス' if b2 < 0 else ''
      print(h,d,b,"->",b2,'(',b1-b2,')',violation_text)
      b1 = b2
  print('------------------------------------')
print('病床マイナス',violation)

In [ ]:
len(set(violation[1]))

# 答えを保存

In [ ]:
db_empty_beds.head(3)

In [ ]:
for h,dates in empty_beds_by_types_optimized_dw[bed_type].items():
  for date,bed in dates.items():
    db_empty_beds.loc[(db_empty_beds['hospital_id']==h)&(db_empty_beds['date']==date),bed_type] = bed
db_empty_beds.to_csv('./output/db_empty_beds_d-wave.csv')

In [ ]:
db_patients.head(3)

In [ ]:
for patient in unsettled_patients_by_next_bed_type:
  db_patients.loc[(db_patients['hospital_id']==patient.current_hospital_id) & (db_patients['patient_id'] == patient.patient_id), 'next_hospital_id'] = patient.next_possible_hospital_ids[0]
db_patients.to_csv('./output/DB_Patients_d-wave.csv')